**In this notebook we test the variation of testing result after varying the input.**

BERT-base-cased is used as the 6 way classifier.

In [1]:
# Importing necessary libraries
import pandas as pd
import numpy as np
from datetime import datetime
import random
import sklearn
import torch
import torch.nn as nn
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

from simpletransformers.classification import ClassificationModel

First we take the Train Set and the Validation set


We randomise their order


we split them into 5 parts (folds) each having the same number of statements in each of the 6 classes.


Eventually 5 transformers will each be trained on 4 of the folds and 1 fold used for training


All 5 transformers will be trained on sets that differ gradually from each other


All will be tested on the same test set

And the result will be examined for the il-posed/ill-conditioned property as described in the thesis report


In [6]:
# procedure for getting the data sets and formatting them for the transformer
 

def prepareDataset( filename):
     
    ReadSet=pd.read_excel(filename )

    ReadSet['text']=ReadSet['Statement']
    ReadSet['labels']=ReadSet['Label']
    
    ReadSet=ReadSet.drop(['ID','Label','Statement','Subject','Speaker','Job','From','Affiliation','PantsTotal','NotRealTotal','BarelyTotal','HalfTotal','MostlyTotal','Truths','Context'
],axis=1)
     
    return ReadSet

In [7]:
# preparing the training dataset
train=prepareDataset( 'train.xlsx')
# and display for inspecting
train

,text,labels
0,The attorney general requires that rape victim...,0
1,President Clinton reduced the scale of our mil...,3
2,"I used tax cuts to help create over 80,000 job...",4
3,"New Mexico moved ""up to"" sixth in the nation i...",4
4,"Corporate profits are up, CEO pay is up, but a...",5
...,...,...
10264,"Under Obamacare, premiums have doubled and tri...",4
10265,We adopted the modern Social Security system a...,5
10266,More than two months ago President Barack Obam...,3
10267,"We had a massive landslide victory, as you kno...",1


In [8]:
# preparing the evaluation/validation dataset
Eval=prepareDataset('valid.xlsx')
# and display for inspecting
Eval

,text,labels
0,The president is brain-dead.,0
1,"Barack Obama supported keeping troops in Iraq,...",3
2,"He's leading by example, refusing contribution...",3
3,I'm the first person who really took up the is...,4
4,I built that border fence in San Diego...and i...,4
...,...,...
1279,CNN accidentally aired 30 minutes of pornograp...,1
1280,President Obamas American Recovery and Reinves...,2
1281,We (in Illinois) have the fifth-highest tax bu...,4
1282,Says Donald Trump won more counties than any c...,4


To keep closer to the standard way of doing K-folds I am including the validation and training set together.

The data will be spilt in 5 folds.  At every trial 4 will be used for training and the other will be used for validation.

In [9]:
train=train.append(Eval, ignore_index = True)
train

,text,labels
0,The attorney general requires that rape victim...,0
1,President Clinton reduced the scale of our mil...,3
2,"I used tax cuts to help create over 80,000 job...",4
3,"New Mexico moved ""up to"" sixth in the nation i...",4
4,"Corporate profits are up, CEO pay is up, but a...",5
...,...,...
11548,CNN accidentally aired 30 minutes of pornograp...,1
11549,President Obamas American Recovery and Reinves...,2
11550,We (in Illinois) have the fifth-highest tax bu...,4
11551,Says Donald Trump won more counties than any c...,4


In [11]:
# preparing the test set dataset
test=prepareDataset('test.xlsx')
test

,text,labels
0,New Mexico was 46th in teacher pay (when he wa...,4
1,Barack Obama and Hillary Clinton have changed ...,3
2,I'll tell you what I can tell this country: If...,1
3,Tommy Thompson created the first school choice...,5
4,Fifty-six percent decline in overall crime. A ...,5
...,...,...
1278,"We have trade agreements with 20 countries, an...",1
1279,On Donald Trumps plan to cut federal funding t...,4
1280,"Black Lives Matter, who are attacking law enfo...",2
1281,Latina who enthusiastically supported Donald T...,0


Now we randomise the order

In [12]:
# first we randomise the order of train

import random
 
randNum=[]

for row in range(len(train)):
    randNum.append(random.random())

train['RandNum']=randNum

train=train.sort_values(by=['RandNum'] )
train=train.drop(['RandNum'],axis=1)
del(randNum)
train
    

,text,labels
3505,The typical white male worker in this country ...,4
5795,Says a pack-a-day smoker who quits because of ...,5
8746,Texas legislation would allow teachers to kill...,3
9477,"30,000 Arizonans may have no choice on Afforda...",4
3919,New energy standards will take away our freedo...,2
...,...,...
699,Warren (Buffett) still does support me.,5
2646,Woonsocket needs a garbage-burning power plant...,0
7423,Ponzi schemer Scott Rothstein gave hundreds of...,3
4933,Says House Republicans spent $50 million on 33...,1


In [13]:
count0=0
count1=0
count2=0
count3=0
count4=0
count5=0

train0=[] #all stamtements that are Pants on Fire  (class 0)
train1=[]
train2=[]
train3=[]
train4=[]
train5=[]


for row in range(len(train)):
        if train.iloc[row,1]==0: 
            count0+=1
            train0.append(train.iloc[row,:])
        if train.iloc[row,1]==1: 
            count1+=1
            train1.append(train.iloc[row,:])
        if train.iloc[row,1]==2: 
            count2+=1
            train2.append(train.iloc[row,:])
        if train.iloc[row,1]==3: 
            count3+=1
            train3.append(train.iloc[row,:])
        if train.iloc[row,1]==4: 
            count4+=1
            train4.append(train.iloc[row,:])
        if train.iloc[row,1]==5: 
            count5+=1
            train5.append(train.iloc[row,:])

print('0s ', count0)
print('1s ', count1)
print('2s ', count2)
print('3s ', count3)
print('4s ', count4)
print('5s ', count5)

            

0s  958
1s  2261
2s  1894
3s  2371
4s  2217
5s  1852


In [14]:
def div5(myinteger):
    size_m5 =myinteger-(myinteger%5)
    QuantityToRemove=size_m5/5
    
    return QuantityToRemove, myinteger%5

C0div5=div5(count0)
C1div5=div5(count1)
C2div5=div5(count2)
C3div5=div5(count3)
C4div5=div5(count4)
C5div5=div5(count5)



print('To omit from class 0s ', C0div5)
print('To omit from class 1s ', C1div5)
print('To omit from class 2s ', C2div5)
print('To omit from class 3s ', C3div5)
print('To omit from class 4s ', C4div5)
print('To omit from class 5s ', C5div5)

To omit from class 0s  (191.0, 3)
To omit from class 1s  (452.0, 1)
To omit from class 2s  (378.0, 4)
To omit from class 3s  (474.0, 1)
To omit from class 4s  (443.0, 2)
To omit from class 5s  (370.0, 2)


In [15]:
train0

[text      Under Obamas plan (for welfare), you wouldnt h...
 labels                                                    0
 Name: 4871, dtype: object,
 text      Technically, (Ted Cruz is) not even an American.
 labels                                                   0
 Name: 8839, dtype: object,
 text      When (Barack Obama) took office, 2.3 million p...
 labels                                                    0
 Name: 7790, dtype: object,
 text      U.S. Reps. Hank Johnson, John Lewis and other ...
 labels                                                    0
 Name: 3363, dtype: object,
 text      There are 30,000 people that have been killed ...
 labels                                                    0
 Name: 3146, dtype: object,
 text      Says Sal Esquivel wants a 5 percent state sale...
 labels                                                    0
 Name: 1909, dtype: object,
 text      Says U.S. Rep. Steve Southerland voted himself...
 labels                                  

 The order is already for each set is already randomised.
We will now, for sets train0 to train5 split each into 5 roughly equal parts.

In [16]:
#train[class][fold]
train01=[]
train02=[]
train03=[]
train04=[]
train05=[] 
   

for row in range(len(train0)):
    
    if row<C0div5[0]: train01.append(train0[row])
        
    if row>=C0div5[0] and row<(C0div5[0]*2): train02.append(train0[row])
        
    if row>=(C0div5[0]*2) and row<(C0div5[0]*3): train03.append(train0[row])
    
    if row>=(C0div5[0]*3) and row<(C0div5[0]*4): train04.append(train0[row])
        
    if row>=(C0div5[0]*4) and row<(C0div5[0]*5): train05.append(train0[row])
        
    if row>=(C0div5[0]*5):
        train01.append(train0[row])
         
     

     



In [17]:
train01

[text      Under Obamas plan (for welfare), you wouldnt h...
 labels                                                    0
 Name: 4871, dtype: object,
 text      Technically, (Ted Cruz is) not even an American.
 labels                                                   0
 Name: 8839, dtype: object,
 text      When (Barack Obama) took office, 2.3 million p...
 labels                                                    0
 Name: 7790, dtype: object,
 text      U.S. Reps. Hank Johnson, John Lewis and other ...
 labels                                                    0
 Name: 3363, dtype: object,
 text      There are 30,000 people that have been killed ...
 labels                                                    0
 Name: 3146, dtype: object,
 text      Says Sal Esquivel wants a 5 percent state sale...
 labels                                                    0
 Name: 1909, dtype: object,
 text      Says U.S. Rep. Steve Southerland voted himself...
 labels                                  

In [18]:
train02

[text      You cannot build a little guy up by tearing a ...
 labels                                                    0
 Name: 11283, dtype: object,
 text      Justice John Roberts Signed Off On Obamas Arre...
 labels                                                    0
 Name: 7233, dtype: object,
 text      Says that President Obama said, Nobody made th...
 labels                                                    0
 Name: 3107, dtype: object,
 text      Says Bruce Springsteen only had 8,000 tickets ...
 labels                                                    0
 Name: 9355, dtype: object,
 text      Says Target installed urinals in a womens bath...
 labels                                                    0
 Name: 9322, dtype: object,
 text      On assault weapon violence: Ducks have more pr...
 labels                                                    0
 Name: 7907, dtype: object,
 text      Obama regime planned the influx of illegal ali...
 labels                               

In [19]:
        
train11=[]
train12=[]
train13=[]
train14=[]
train15=[]
train1remaining=[]

train21=[]
train22=[]
train23=[]
train24=[]
train25=[]
train2remaining=[]

train31=[]
train32=[]
train33=[]
train34=[]
train35=[]
train3remaining=[]

train41=[]
train42=[]
train43=[]
train44=[]
train45=[]
train4remaining=[]

train51=[]
train52=[]
train53=[]
train54=[]
train55=[]
train5remaining=[]



In [20]:

for row in range(len(train1)):
    if row<C1div5[0]: train11.append(train1[row])
        
    if row>=C1div5[0] and row<(C1div5[0]*2): train12.append(train1[row])
        
    if row>=(C1div5[0]*2) and row<(C1div5[0]*3): train13.append(train1[row])
    
    if row>=(C1div5[0]*3) and row<(C1div5[0]*4): train14.append(train1[row])
        
    if row>=(C1div5[0]*4) and row<(C1div5[0]*5): train15.append(train1[row])
        
    if row>=(C1div5[0]*5):
        train11.append(train1[row])
         



In [21]:
for row in range(len(train2)):
    if row<C2div5[0]: train21.append(train2[row])
        
    if row>=C2div5[0] and row<(C2div5[0]*2): train22.append(train2[row])
        
    if row>=(C2div5[0]*2) and row<(C2div5[0]*3): train23.append(train2[row])
    
    if row>=(C2div5[0]*3) and row<(C2div5[0]*4): train24.append(train2[row])
        
    if row>=(C2div5[0]*4) and row<(C2div5[0]*5): train25.append(train2[row])
        
    if row>=(C2div5[0]*5):
        train21.append(train2[row])
         


In [22]:

for row in range(len(train3)):
    if row<C3div5[0]: train31.append(train3[row])
        
    if row>=C3div5[0] and row<(C3div5[0]*2): train32.append(train3[row])
        
    if row>=(C3div5[0]*2) and row<(C3div5[0]*3): train33.append(train3[row])
    
    if row>=(C3div5[0]*3) and row<(C3div5[0]*4): train34.append(train3[row])
        
    if row>=(C3div5[0]*4) and row<(C3div5[0]*5): train35.append(train3[row])
        
    if row>=(C3div5[0]*5):
        train31.append(train3[row])
         

In [23]:
for row in range(len(train4)):
    if row<C4div5[0]: train41.append(train4[row])
        
    if row>=C4div5[0] and row<(C4div5[0]*2): train42.append(train4[row])
        
    if row>=(C4div5[0]*2) and row<(C4div5[0]*3): train43.append(train4[row])
    
    if row>=(C4div5[0]*3) and row<(C4div5[0]*4): train44.append(train4[row])
        
    if row>=(C4div5[0]*4) and row<(C4div5[0]*5): train45.append(train4[row])
        
    if row>=(C4div5[0]*5):
        train41.append(train4[row])
         

In [24]:

for row in range(len(train5)):
    if row<C5div5[0]: train51.append(train5[row])
        
    if row>=C5div5[0] and row<(C5div5[0]*2): train52.append(train5[row])
        
    if row>=(C5div5[0]*2) and row<(C5div5[0]*3): train53.append(train5[row])
    
    if row>=(C5div5[0]*3) and row<(C5div5[0]*4): train54.append(train5[row])
        
    if row>=(C5div5[0]*4) and row<(C5div5[0]*5): train55.append(train5[row])
        
    if row>=(C5div5[0]*5):
        train51.append(train5[row])
         


In [25]:
pants1=pd.DataFrame(train01, columns=['text','labels'])
pants2=pd.DataFrame(train02, columns=['text','labels'])
pants3=pd.DataFrame(train03, columns=['text','labels'])
pants4=pd.DataFrame(train04, columns=['text','labels'])
pants5=pd.DataFrame(train05, columns=['text','labels'])


fake1=pd.DataFrame(train11, columns=['text','labels'])
fake2=pd.DataFrame(train12, columns=['text','labels'])
fake3=pd.DataFrame(train13, columns=['text','labels'])
fake4=pd.DataFrame(train14, columns=['text','labels'])
fake5=pd.DataFrame(train15, columns=['text','labels'])


Mfake1=pd.DataFrame(train21, columns=['text','labels'])
Mfake2=pd.DataFrame(train22, columns=['text','labels'])
Mfake3=pd.DataFrame(train23, columns=['text','labels'])
Mfake4=pd.DataFrame(train24, columns=['text','labels'])
Mfake5=pd.DataFrame(train25, columns=['text','labels'])


half1=pd.DataFrame(train31, columns=['text','labels'])
half2=pd.DataFrame(train32, columns=['text','labels'])
half3=pd.DataFrame(train33, columns=['text','labels'])
half4=pd.DataFrame(train34, columns=['text','labels'])
half5=pd.DataFrame(train35, columns=['text','labels'])


Mreal1=pd.DataFrame(train41, columns=['text','labels'])
Mreal2=pd.DataFrame(train42, columns=['text','labels'])
Mreal3=pd.DataFrame(train43, columns=['text','labels'])
Mreal4=pd.DataFrame(train44, columns=['text','labels'])
Mreal5=pd.DataFrame(train45, columns=['text','labels'])


real1=pd.DataFrame(train51, columns=['text','labels'])
real2=pd.DataFrame(train52, columns=['text','labels'])
real3=pd.DataFrame(train53, columns=['text','labels'])
real4=pd.DataFrame(train54, columns=['text','labels'])
real5=pd.DataFrame(train55, columns=['text','labels'])







frames1 = [pants2, pants3, pants4, pants5, fake2, fake3, fake4, fake5, Mfake2, Mfake3, Mfake4, Mfake5, half2, half3, half4, half5, Mreal2, Mreal3, Mreal4, Mreal5, real2, real3, real4, real5 ]

frames2 = [ pants1, pants3, pants4, pants5,fake1,  fake3, fake4, fake5,Mfake1, Mfake3, Mfake4, Mfake5,half1, half3, half4, half5,Mreal1, Mreal3, Mreal4, Mreal5,real1, real3, real4, real5 ]


frames3 = [pants1, pants2,  pants4, pants5,fake1, fake2, fake4, fake5,Mfake1, Mfake2, Mfake4, Mfake5,half1, half2,  half4, half5,Mreal1, Mreal2,  Mreal4, Mreal5,real1, real2,  real4, real5]


frames4 = [ pants1, pants2, pants3, pants5,fake1, fake2, fake3, fake5,Mfake1, Mfake2, Mfake3, Mfake5,half1, half2, half3, half5,Mreal1, Mreal2, Mreal3,  Mreal5,real1, real2, real3,  real5 ]

frames5 = [pants1, pants2, pants3, pants4, fake1, fake2, fake3, fake4,Mfake1, Mfake2, Mfake3, Mfake4, half1, half2, half3, half4,Mreal1, Mreal2, Mreal3, Mreal4, real1, real2, real3, real4]




train_fold1 = pd.concat(frames1)
train_fold2 = pd.concat(frames2)
train_fold3 = pd.concat(frames3)
train_fold4 = pd.concat(frames4)
train_fold5 = pd.concat(frames5)

#we set the omitted fold as the validation set

frames1=[pants1,fake1,Mfake1,half1,Mreal1,real1]
valid1 = pd.concat(frames1)

frames2=[pants2,fake2,Mfake2,half2,Mreal2,real2]
valid2 = pd.concat(frames2)

frames3=[pants3,fake3,Mfake3,half3,Mreal3,real3]
valid3 = pd.concat(frames3)

frames4=[pants4,fake4,Mfake4,half4,Mreal4,real4]
valid4 = pd.concat(frames4)

frames5=[pants5,fake5,Mfake5,half5,Mreal5,real5]
valid5 = pd.concat(frames5)
 

In [26]:
def randomiseSet(set):
    #this function randomises the order of the set.
    #order shouldn't be an issue but better keep things close to the realistic realms
 
    randNum=[]

    for row in range(len(set)):
        randNum.append(random.random())

    set['RandNum']=randNum
    set=set.sort_values(by=['RandNum'] )
    set=set.drop(['RandNum'],axis=1)
    
    return set

In [27]:
train_fold1=randomiseSet(train_fold1)
train_fold2=randomiseSet(train_fold2)
train_fold3=randomiseSet(train_fold3)
train_fold4=randomiseSet(train_fold4)
train_fold5=randomiseSet(train_fold5)
valid1=randomiseSet(valid1)
valid2=randomiseSet(valid2)
valid3=randomiseSet(valid3)
valid4=randomiseSet(valid4)
valid5=randomiseSet(valid5)

In [28]:
train_fold1.to_excel('./folds/train_fold1.xls')
train_fold2.to_excel('./folds/train_fold2.xls')
train_fold3.to_excel('./folds/train_fold3.xls')
train_fold4.to_excel('./folds/train_fold4.xls')
train_fold5.to_excel('./folds/train_fold5.xls')

valid1.to_excel('./folds/valid1.xls')
valid2.to_excel('./folds/valid2.xls')
valid3.to_excel('./folds/valid3.xls')
valid4.to_excel('./folds/valid4.xls')
valid5.to_excel('./folds/valid5.xls')

# We can now run the tests
 
 

In [30]:
#Set the model being used here
model_class='bert'  # bert or roberta or albert
model_version='bert-base-cased' #bert-base-cased, roberta-base, roberta-large, albert-base-v2 OR albert-large-v2
labels_count=6  # the number of classification classes


# Importing necessary libraries
import pandas as pd
import numpy as np
from datetime import datetime
import random
import sklearn
import torch
import torch.nn as nn
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

from simpletransformers.classification import ClassificationModel

 
# and get the test set ready
def prepareDataset( filename):
     
    ReadSet=pd.read_excel(filename )

    ReadSet['text']=ReadSet['Statement']
    ReadSet['labels']=ReadSet['Label']
    
    ReadSet=ReadSet.drop(['ID','Label','Statement','Subject','Speaker','Job','From','Affiliation','PantsTotal','NotRealTotal','BarelyTotal','HalfTotal','MostlyTotal','Truths','Context'
],axis=1)
     
    return ReadSet



# preparing the test set dataset
test=prepareDataset('test.xlsx')



## Fold1 training and capturing predictions

In [31]:
fold_number='1'

train=pd.read_excel('./folds/train_fold'+fold_number+'.xls')
Eval =pd.read_excel('./folds/valid'+fold_number+'.xls') #evaluation set


In [32]:

output_folder='./folds/fold'+fold_number+'/'+model_class+'/'+model_version+"/"
cache_directory= "./folds/fold"+fold_number+'/'+model_class+"/"+model_version+"/cache/"


print('model variables were set up: ')

 
save_every_steps=1285
# assuming training batch size of 8
# any number above 1284 saves the model only at every epoch
# Saving the model mid training very often will consume disk space fast

train_args={
    "output_dir":output_folder,
    "cache_dir":cache_directory,
    'reprocess_input_data': True,
    'overwrite_output_dir': True,
    'num_train_epochs': 2,
    "save_steps": save_every_steps, 
    "learning_rate": 2e-5,
    "train_batch_size": 64,
    "eval_batch_size": 16,
    "evaluate_during_training_steps": 5,
    "max_seq_length": 100,
    "n_gpu": 1,
}

# Create a ClassificationModel
model = ClassificationModel(model_class, model_version, num_labels=labels_count, args=train_args) 

# loading a previously saved ClassificationModel model based on this particular Transformer Class and model_name



model variables were set up: 


In [33]:
# loading the checkpoint that gave the best result
'''
CheckPoint='checkpoint-254-epoch-2'   


preSavedCheckpoint=output_folder+CheckPoint

print('Loading model, please wait...')
model = ClassificationModel( model_class, preSavedCheckpoint, num_labels=labels_count, args=train_args) 
print('model in use is :', preSavedCheckpoint )
''' 

"\nCheckPoint='checkpoint-254-epoch-2'   \n\n\npreSavedCheckpoint=output_folder+CheckPoint\n\nprint('Loading model, please wait...')\nmodel = ClassificationModel( model_class, preSavedCheckpoint, num_labels=labels_count, args=train_args) \nprint('model in use is :', preSavedCheckpoint )\n"

In [34]:
# Train the model
current_time = datetime.now()
model.train_model(train)
print("Training time taken: ", datetime.now() - current_time, ' at:',datetime.now())

Converting to features started. Cache is not used.



Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic


Running loss: 1.758073

C:\Users\Mark\Anaconda3\envs\thesis\lib\site-packages\torch\optim\lr_scheduler.py:110: UserWarning: Seems like `optimizer.step()` has been overridden after learning rate scheduler initialization. Please, make sure to call `optimizer.step()` before `lr_scheduler.step()`. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Seems like `optimizer.step()` has been overridden after learning rate scheduler "


Running loss: 1.764588

C:\Users\Mark\Anaconda3\envs\thesis\lib\site-packages\torch\optim\lr_scheduler.py:224: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


Running loss: 1.667812


Running loss: 1.741215

Training of bert model complete. Saved to ./folds/fold1/bert/bert-base-cased/.
Training time taken:  0:05:42.854016  at: 2020-04-29 12:37:23.178257


In [35]:
TrainResult, TrainModel_outputs, wrong_predictions = model.eval_model(train_fold1, acc=sklearn.metrics.accuracy_score)
 
EvalResult, EvalModel_outputs, wrong_predictions = model.eval_model(Eval, acc=sklearn.metrics.accuracy_score)

TestResult, TestModel_outputs, wrong_predictions = model.eval_model(test, acc=sklearn.metrics.accuracy_score)

print('Training Result:', TrainResult['acc'])
#print('Model Out:', TrainModel_outputs)

print('Eval Result:', EvalResult['acc'])
#print('Model Out:', EvalModel_outputs)

print('Test Set Result:', TestResult['acc'])
#print('Model Out:', TestModel_outputs)

Converting to features started. Cache is not used.



{'mcc': 0.17415988348261832, 'acc': 0.327448006932409, 'eval_loss': 1.6099405352742875}
Converting to features started. Cache is not used.



{'mcc': 0.10580513101737966, 'acc': 0.27401981904351574, 'eval_loss': 1.6815254100381511}
Converting to features started. Cache is not used.



{'mcc': 0.12267236840710048, 'acc': 0.289166017147311, 'eval_loss': 1.6654093397988214}
Training Result: 0.327448006932409
Eval Result: 0.27401981904351574
Test Set Result: 0.289166017147311


In [36]:
Pred=[]

countCorrect=0

for row in range(TestModel_outputs.shape[0]):
    outputs=TestModel_outputs[row]
    #print(test.iloc[row,0])
    print(outputs, end=' ')
    
    result=0
    if outputs[0]<outputs[1]:result=1
    if outputs[result]<outputs[2]:result=2
    if outputs[result]<outputs[3]:result=3
    if outputs[result]<outputs[4]:result=4
    if outputs[result]<outputs[5]:result=5
    Pred.append(result)
    print(result, ' ',test.iloc[row,1], end=' ')
    if result==test.iloc[row,1]:
        countCorrect+=1
        print('Match',countCorrect)
    print('')

print(countCorrect)
#Pred

[-1.5898438  -0.06109619 -0.31079102  0.5805664   0.78125     0.70654297] 4   4 Match 1

[-0.36523438  0.4165039   0.21862793  0.18188477 -0.25390625 -0.5761719 ] 1   3 
[-0.64746094  0.44604492  0.08508301 -0.00604248  0.00473404 -0.0418396 ] 1   1 Match 2

[-1.0410156   0.07019043  0.0670166   0.37329102  0.171875    0.01812744] 3   5 
[-1.4541016  -0.40356445 -0.17590332  0.765625    0.6958008   0.48608398] 3   5 
[-1.015625    0.32226562 -0.03143311  0.31298828  0.47875977  0.70751953] 5   2 
[-1.6572266  -0.15539551 -0.07177734  0.7675781   0.79833984  0.44384766] 4   4 Match 3

[-1.2978516  -0.02947998 -0.03405762  0.6245117   0.89404297  0.68115234] 4   5 
[-1.3730469   0.01867676 -0.13354492  0.38671875  0.8222656   0.84716797] 5   4 
[-1.2304688  -0.05096436 -0.1192627   0.41357422  0.69189453  0.5517578 ] 4   5 
[-0.1328125   0.4555664   0.3959961   0.09753418 -0.4182129  -0.60253906] 1   5 
[-1.1601562   0.21447754  0.1081543   0.3876953   0.3737793   0.05328369] 3   3 Match

[-1.4589844  -0.01010132  0.17468262  0.7763672   0.72998047  0.31079102] 3   4 
[-1.0878906  -0.17260742 -0.13232422  0.18115234  0.4033203   0.33740234] 4   5 
[-1.3369141   0.14660645  0.05215454  0.58203125  0.6435547   0.36767578] 4   3 
[-0.31518555  0.23864746  0.08154297 -0.06347656 -0.42773438 -0.5175781 ] 1   3 
[-0.53125     0.4321289   0.3059082   0.2388916  -0.22766113 -0.34960938] 1   1 Match 58

[-0.7495117   0.6191406   0.20751953  0.26245117 -0.1262207  -0.2998047 ] 1   3 
[-0.36401367  0.97509766 -0.39379883 -0.04803467 -0.5292969  -0.3269043 ] 1   3 
[ 0.22509766  0.44604492  0.31469727 -0.38232422 -0.6723633  -0.8173828 ] 1   0 
[-0.5644531   0.42236328  0.42016602  0.17834473 -0.20825195 -0.5625    ] 1   2 
[-0.04766846  0.33935547  0.18164062 -0.0758667  -0.4777832  -0.6923828 ] 1   5 
[-0.36157227  0.40600586  0.34399414  0.21289062 -0.37475586 -0.85253906] 1   2 
[-1.4082031  -0.09606934 -0.23754883  0.64941406  0.8286133   0.6821289 ] 4   4 Match 59

[ 0.414794

[-1.0097656   0.24511719 -0.0892334   0.1229248   0.47607422  0.53564453] 5   5 Match 89

[-1.2392578   0.35839844  0.28833008  0.47436523  0.24401855 -0.14404297] 3   1 
[-1.1855469   0.1072998  -0.01470947  0.31152344  0.5932617   0.5449219 ] 4   5 
[-0.4350586   0.26586914  0.29589844  0.065979   -0.32226562 -0.5917969 ] 2   4 
[-1.6083984  -0.04931641 -0.14428711  0.51220703  0.61621094  0.52783203] 4   3 
[-0.7871094   0.4038086   0.12524414  0.4387207   0.19299316 -0.30908203] 3   2 
[-0.38012695  0.22851562 -0.0279541  -0.19763184 -0.24450684 -0.20996094] 1   0 
[ 0.0489502   0.29882812  0.13476562 -0.35107422 -0.45922852 -0.5473633 ] 1   1 Match 90

[-1.1669922   0.14831543 -0.12469482  0.29101562  0.42895508  0.41796875] 4   5 
[-0.98095703  0.36791992  0.07287598  0.33984375  0.46411133  0.29125977] 4   3 
[-1.3164062  -0.05804443  0.03085327  0.46264648  0.546875    0.45458984] 4   4 Match 91

[-0.41015625  0.3713379   0.38916016 -0.1385498  -0.49023438 -0.68408203] 2   1 
[

[-1.1982422   0.05175781  0.2368164   0.6611328   0.20239258 -0.20483398] 3   3 Match 130

[-1.3769531  -0.11273193 -0.04873657  0.5488281   0.65283203  0.27416992] 4   2 
[-1.125       0.15344238  0.19506836  0.5961914   0.16809082 -0.10339355] 3   4 
[-1.2949219   0.08770752  0.11157227  0.66503906  0.19824219 -0.09472656] 3   3 Match 131

[-0.08966064  0.21032715  0.13757324 -0.12597656 -0.53759766 -0.46191406] 1   4 
[-0.87060547  0.0552063   0.16833496  0.35205078  0.2890625  -0.02629089] 3   0 
[-1.5136719  -0.16955566 -0.09967041  0.7763672   0.9301758   0.5488281 ] 4   5 
[-1.34375    -0.04110718  0.18688965  0.7011719   0.5957031   0.30932617] 3   5 
[-0.5317383   0.3708496   0.4777832   0.17443848 -0.38378906 -0.57128906] 2   2 Match 132

[-1.0722656   0.28344727  0.21875     0.3552246  -0.18017578 -0.49243164] 3   1 
[-1.4189453   0.13391113 -0.14819336  0.5332031   0.7421875   0.49072266] 4   4 Match 133

[-0.6767578   0.20129395  0.19189453  0.2010498  -0.03588867 -0.03102

[-1.5800781   0.08520508  0.1784668   0.6513672   0.5786133   0.30541992] 3   5 
[-1.4726562  -0.21118164 -0.21057129  0.61816406  0.8779297   0.6791992 ] 4   4 Match 175

[-1.1953125  -0.08764648  0.2939453   0.61083984  0.17224121 -0.01443481] 3   2 
[-0.10272217  0.44970703  0.36669922 -0.5332031  -0.64697266 -0.578125  ] 1   1 Match 176

[-0.3305664   0.41333008  0.14978027 -0.27563477 -0.4543457  -0.5620117 ] 1   3 
[-1.0527344   0.16259766  0.3720703   0.64160156 -0.04742432 -0.3623047 ] 3   1 
[ 0.25146484  0.47631836  0.2446289  -0.23718262 -0.66308594 -0.89160156] 1   1 Match 177

[-0.29760742  0.41552734 -0.17895508 -0.3955078  -0.43188477 -0.18920898] 1   2 
[-1.1552734   0.14489746  0.15661621  0.28979492  0.32055664  0.3083496 ] 4   3 
[-0.65234375  0.4296875   0.24597168  0.26489258 -0.22229004 -0.4567871 ] 1   3 
[-0.62646484  0.24731445  0.30908203  0.48535156 -0.16674805 -0.4140625 ] 3   2 
[-1.4375      0.06420898 -0.12963867  0.48901367  0.9042969   0.6533203 ] 4   2

...

[-0.82958984  0.07318115  0.07806396  0.24377441  0.10699463  0.00443268] 3   4 
[-1.4306641  -0.03564453 -0.26611328  0.65478516  1.1279297   0.85498047] 4   1 
[-1.3886719  -0.28979492 -0.2902832   0.58740234  0.8730469   0.56884766] 4   3 
[-0.61572266  0.10192871 -0.26171875 -0.1307373  -0.04660034 -0.01754761] 1   4 
[ 0.48828125  0.49438477  0.44335938 -0.4873047  -0.7709961  -1.0273438 ] 1   0 
[-1.3583984   0.12194824 -0.38793945  0.41992188  0.8984375   0.8754883 ] 4   2 
[-1.0820312   0.24926758 -0.22424316  0.22583008  0.58447266  0.38378906] 4   1 
[-1.0214844   0.35302734 -0.07861328  0.16564941  0.29516602  0.40698242] 5   5 Match 240

[-1.4169922  -0.27905273  0.04049683  0.87597656  0.5292969   0.1784668 ] 3   2 
[-1.4541016  -0.15698242 -0.25561523  0.69384766  1.0029297   0.7792969 ] 4   4 Match 241

[-1.6015625  -0.21130371 -0.11688232  0.84765625  0.68310547  0.33447266] 3   3 Match 242

[-1.2568359  -0.12097168  0.02252197  0.47705078  0.6772461   0.4897461 ] 4   4

[ 0.19958496  0.4501953  -0.00674438 -0.28076172 -0.7055664  -0.8413086 ] 1   0 
[-1.2753906   0.18115234  0.17492676  0.6098633   0.5         0.22424316] 3   4 
[-1.2978516   0.17456055  0.24658203  0.61865234  0.28881836 -0.01991272] 3   2 
[-0.66259766  0.2692871   0.37182617  0.3059082  -0.21728516 -0.39892578] 2   2 Match 269

[-1.0341797   0.28051758  0.23657227  0.4272461   0.4104004   0.26757812] 3   2 
[-1.4111328  -0.12463379  0.08642578  0.64404297  0.5332031   0.28979492] 3   4 
[-0.67041016  0.2890625  -0.07818604  0.09680176 -0.09246826 -0.34179688] 1   5 
[-1.0566406   0.09069824  0.11297607  0.46606445  0.2919922   0.08044434] 3   4 
[ 0.3474121   0.4658203   0.29492188 -0.43701172 -0.7895508  -0.9897461 ] 1   2 
[-0.83251953  0.0894165   0.23913574  0.46069336 -0.00774384 -0.11621094] 3   4 
[-1.4082031  -0.35913086 -0.03634644  0.9326172   0.70166016  0.45898438] 3   4 
[-0.58984375  0.4638672   0.15771484  0.05334473 -0.08459473 -0.05944824] 1   2 
[-0.4699707   0.27

[-1.5019531  -0.28808594 -0.30297852  0.88134766  0.9868164   0.83203125] 4   3 
[-1.59375     0.05178833 -0.30322266  0.24633789  0.62158203  0.5595703 ] 4   5 
[-0.61083984  0.94433594 -0.30371094  0.11248779 -0.3840332  -0.17834473] 1   1 Match 301

[-0.67089844  0.13000488  0.19018555  0.32666016  0.13085938 -0.20739746] 3   1 
[-1.6123047  -0.04647827 -0.23498535  0.6459961   0.91503906  0.7050781 ] 4   5 
[ 0.18103027  0.36132812  0.6201172  -0.23400879 -0.5786133  -0.79003906] 2   3 
[-1.2880859  -0.07196045 -0.28564453  0.34155273  0.6376953   0.51708984] 4   1 
[-1.3164062  -0.11560059  0.06445312  0.7685547   0.48266602  0.08178711] 3   2 
[-1.3261719   0.12658691 -0.16259766  0.37670898  0.5566406   0.5253906 ] 4   4 Match 302

[ 0.13183594  0.35961914 -0.06921387 -0.3762207  -0.6645508  -0.55859375] 1   1 Match 303

[-0.16369629  0.3984375   0.23474121  0.09698486 -0.28808594 -0.5546875 ] 1   1 Match 304

[ 0.06555176  0.33984375  0.36621094 -0.03823853 -0.4736328  -0.90185

[-1.5458984  -0.3071289  -0.45996094  0.7529297   1.0283203   0.84228516] 4   0 
[-1.2929688  -0.04891968 -0.3618164   0.2397461   0.7709961   0.8198242 ] 5   3 
[-1.2783203   0.05755615 -0.105896    0.57470703  0.8129883   0.64746094] 4   5 
[-1.203125    0.0198822  -0.29077148  0.44799805  0.7919922   0.78222656] 4   2 
[ 0.49560547  0.4404297   0.2775879  -0.39892578 -0.69091797 -0.8095703 ] 0   3 
[-0.5410156   0.36694336  0.19824219  0.14416504 -0.12512207 -0.43603516] 1   4 
[-1.2138672  -0.18469238 -0.06866455  0.4387207   0.66748047  0.6254883 ] 4   5 
[-1.1425781  -0.05160522  0.00229645  0.34545898  0.50146484  0.5234375 ] 5   3 
[-0.19067383  0.44335938  0.11975098 -0.20422363 -0.42651367 -0.46142578] 1   0 
[-0.7729492   0.1505127   0.40063477  0.47705078 -0.05444336 -0.42797852] 3   2 
[-0.5654297   0.140625    0.23986816  0.29125977  0.00183773 -0.1907959 ] 3   1 
[-0.9194336   0.16101074  0.11120605  0.36645508  0.35620117  0.30688477] 3   2 
[-0.4165039   0.22070312  0.

[ 0.27539062  0.5385742   0.4567871  -0.24255371 -0.5761719  -0.9980469 ] 1   3 
[-0.58447266  0.11895752  0.28051758  0.28759766 -0.26489258 -0.5732422 ] 3   2 
[ 0.03417969  0.23388672  0.22692871 -0.29345703 -0.56591797 -0.67041016] 1   2 
[-1.2333984   0.08068848  0.20996094  0.61279297  0.45776367  0.02429199] 3   1 
[-0.859375    0.35839844 -0.11077881  0.01766968 -0.06994629 -0.01087189] 1   1 Match 362

[-0.68066406  0.56347656  0.01832581  0.11480713  0.0463562  -0.00582504] 1   3 
[-0.47338867  0.40014648  0.17419434  0.03170776 -0.1472168  -0.4560547 ] 1   5 
[ 0.1505127   0.46826172  0.2993164  -0.43310547 -0.5097656  -0.7138672 ] 1   4 
[-1.1474609  -0.07287598 -0.20129395  0.22216797  0.5073242   0.28320312] 4   0 
[-1.2080078   0.32299805  0.0380249   0.40478516  0.24450684 -0.04336548] 3   2 
[-1.4023438  -0.3684082  -0.33032227  0.65625     0.8647461   0.5571289 ] 4   5 
[-1.1386719  -0.04150391  0.14990234  0.57666016  0.49316406  0.2644043 ] 3   4 
[-0.6821289   0.44

In [37]:
from sklearn import metrics
 
print(metrics.confusion_matrix(test['labels'],Pred))

[[  4  50   9  16  11   2]
 [  1 114  12  56  48  19]
 [  2  63  30  66  42  11]
 [  2  74  20  92  67  12]
 [  2  48   5  72 105  17]
 [  1  45   7  48  84  26]]


In [38]:
target_names = ['Pants', 'False', 'Barely-True','Half-True','Mostly-True','True']

print(metrics.classification_report(test['labels'], Pred,target_names =target_names))

              precision    recall  f1-score   support

       Pants       0.33      0.04      0.08        92
       False       0.29      0.46      0.35       250
 Barely-True       0.36      0.14      0.20       214
   Half-True       0.26      0.34      0.30       267
 Mostly-True       0.29      0.42      0.35       249
        True       0.30      0.12      0.17       211

    accuracy                           0.29      1283
   macro avg       0.31      0.25      0.24      1283
weighted avg       0.30      0.29      0.27      1283



In [39]:
Fold1_Predictions=pd.DataFrame(Pred, columns=['Pred1'])
Fold1_Predictions

,Pred1
0,4
1,1
2,1
3,3
4,3
...,...
1278,4
1279,1
1280,0
1281,1


In [40]:
Fold1_Predictions.to_excel(output_folder+'/Saves/fold1_Predictions.xls')

In [41]:
#clearing GPU cache

del(model)
del(TrainResult, TrainModel_outputs, EvalResult, EvalModel_outputs, TestResult, TestModel_outputs, wrong_predictions)
torch.cuda.empty_cache()

## Fold 2: training & caturing predictions

In [42]:
fold_number='2'

train=pd.read_excel('./folds/train_fold'+fold_number+'.xls')
Eval=pd.read_excel('./folds/valid'+fold_number+'.xls') #evaluation set


In [55]:

output_folder='./folds/fold'+fold_number+'/'+model_class+'/'+model_version+"/"
cache_directory= "./folds/fold"+fold_number+'/'+model_class+"/"+model_version+"/cache/"


print('model variables were set up: ')

 
save_every_steps=1285
# assuming training batch size of 8
# any number above 1284 saves the model only at every epoch
# Saving the model mid training very often will consume disk space fast

train_args={
    "output_dir":output_folder,
    "cache_dir":cache_directory,
    'reprocess_input_data': True,
    'overwrite_output_dir': True,
    'num_train_epochs': 1,
    "save_steps": save_every_steps, 
    "learning_rate": 2e-5,
    "train_batch_size": 64,
    "eval_batch_size": 16,
    "evaluate_during_training_steps": 312,
    "max_seq_length": 100,
    "n_gpu": 1,
}

# Create a ClassificationModel
model = ClassificationModel(model_class, model_version, num_labels=labels_count, args=train_args) 

model variables were set up: 


In [56]:
# loading the checkpoint that gave the best result

CheckPoint='checkpoint-145-epoch-1'  #epoch 1


preSavedCheckpoint=output_folder+CheckPoint

print('Loading model, please wait...')
model = ClassificationModel( model_class, preSavedCheckpoint, num_labels=labels_count, args=train_args) 
print('model in use is :', preSavedCheckpoint )


Loading model, please wait...
model in use is : ./folds/fold2/bert/bert-base-cased/checkpoint-145-epoch-1


In [57]:
# Train the model
current_time = datetime.now()
model.train_model(train)
print("Training time: ", datetime.now() - current_time)

Converting to features started. Cache is not used.



Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic


Running loss: 1.592071

Training of bert model complete. Saved to ./folds/fold2/bert/bert-base-cased/.
Training time:  0:03:15.332954


In [58]:
TrainResult, TrainModel_outputs, wrong_predictions = model.eval_model(train, acc=sklearn.metrics.accuracy_score)
 
EvalResult, EvalModel_outputs, wrong_predictions = model.eval_model(Eval, acc=sklearn.metrics.accuracy_score)

TestResult, TestModel_outputs, wrong_predictions = model.eval_model(test, acc=sklearn.metrics.accuracy_score)

print('Training Result:', TrainResult['acc'])
#print('Model Out:', TrainModel_outputs)

print('Eval Result:', EvalResult['acc'])
#print('Model Out:', EvalModel_outputs)

print('Test Set Result:', TestResult['acc'])
#print('Model Out:', TestModel_outputs)

Features loaded from cache at ./folds/fold2/bert/bert-base-cased/cache/cached_dev_bert_100_6_9245



{'mcc': 0.19515031150646422, 'acc': 0.3442942130881558, 'eval_loss': 1.5801145473566023}
Features loaded from cache at ./folds/fold2/bert/bert-base-cased/cache/cached_dev_bert_100_6_2308



{'mcc': 0.08563514355025141, 'acc': 0.2582322357019064, 'eval_loss': 1.6800497778530779}
Features loaded from cache at ./folds/fold2/bert/bert-base-cased/cache/cached_dev_bert_100_6_1283



{'mcc': 0.0890518270208055, 'acc': 0.2626656274356976, 'eval_loss': 1.664914247430401}
Training Result: 0.3442942130881558
Eval Result: 0.2582322357019064
Test Set Result: 0.2626656274356976


In [59]:
Pred=[]

countCorrect=0

for row in range(TestModel_outputs.shape[0]):
    outputs=TestModel_outputs[row]
    #print(test.iloc[row,0])
    print(outputs, end=' ')
    
    result=0
    if outputs[0]<outputs[1]:result=1
    if outputs[result]<outputs[2]:result=2
    if outputs[result]<outputs[3]:result=3
    if outputs[result]<outputs[4]:result=4
    if outputs[result]<outputs[5]:result=5
    Pred.append(result)
    print(result, ' ',test.iloc[row,1], end=' ')
    if result==test.iloc[row,1]:
        countCorrect+=1
        print('Match',countCorrect)
    print('')

print(countCorrect)
#Pred

[-1.5546875  -0.23657227 -0.3623047   0.2626953   1.2246094   1.0800781 ] 4   4 Match 1

[-1.0341797   0.5097656  -0.27685547  0.16259766  0.16516113  0.38891602] 1   3 
[-0.9838867   0.48266602 -0.037323    0.24035645  0.17810059 -0.01399994] 1   1 Match 2

[-0.53466797  0.05020142  0.39648438  0.49291992  0.09503174 -0.22583008] 3   5 
[-1.8544922  -0.06646729 -0.3642578   0.4951172   1.1210938   0.6147461 ] 4   5 
[-1.4248047   0.04702759 -0.36865234  0.5966797   0.6879883   0.79541016] 5   2 
[-1.8144531  -0.01966858 -0.25317383  0.56689453  0.9819336   0.6875    ] 4   4 Match 3

[-1.4082031   0.05941772  0.05593872  0.3798828   0.6323242   0.26538086] 4   5 
[-1.4033203  -0.00526428 -0.45263672  0.10809326  0.86035156  1.0996094 ] 5   4 
[-0.8959961   0.17797852  0.00176811  0.49902344  0.26904297 -0.18908691] 3   5 
[-0.45874023  0.21325684  0.45288086  0.5180664  -0.19042969 -0.49780273] 3   5 
[-1.1367188   0.42163086  0.09875488  0.48266602  0.37451172 -0.07531738] 3   3 Match


[-1.7607422   0.00263977 -0.08422852  0.5488281   0.86621094  0.45117188] 4   3 
[-1.1074219e+00  2.3303223e-01  3.5095215e-04  3.6230469e-01
  4.3457031e-01  1.3110352e-01] 4   2 
[-0.79052734  0.5546875   0.12280273  0.5732422  -0.01049042 -0.39038086] 3   2 
[-1.8046875  -0.1776123  -0.4958496   0.52001953  0.97216797  0.92871094] 4   4 Match 31

[-1.609375    0.03543091 -0.19970703  0.453125    0.80908203  0.61279297] 4   4 Match 32

[-0.7871094   0.36767578  0.41308594  0.5595703   0.13342285 -0.24047852] 3   5 
[-0.33740234  0.50097656  0.2590332   0.15527344 -0.15649414 -0.47558594] 1   1 Match 33

[-0.9248047   0.49658203 -0.09222412  0.33203125  0.29248047  0.13757324] 1   1 Match 34

[-1.4794922   0.08312988  0.13049316  0.8300781   0.47583008  0.07220459] 3   4 
[-1.7226562  -0.12384033 -0.23632812  0.63623047  0.8745117   0.6972656 ] 4   5 
[-1.7216797   0.08166504 -0.19018555  0.6142578   0.76953125  0.25268555] 4   1 
[-1.6962891  -0.13647461  0.03820801  0.953125    0.7

[-1.1435547   0.07409668  0.26904297  0.78125     0.2163086  -0.08416748] 3   3 Match 66

[ 0.26831055  0.71435547  0.34448242 -0.12359619 -0.5419922  -0.71777344] 1   2 
[ 0.15966797  0.6845703   0.5         0.12249756 -0.5185547  -0.84814453] 1   2 
[ 0.14172363  0.7133789   0.2536621  -0.20019531 -0.45629883 -0.60009766] 1   5 
[ 0.13659668  0.58984375  0.4243164   0.04049683 -0.44604492 -0.91064453] 1   3 
[-1.2880859  -0.00555801  0.08618164  0.6225586   0.4267578   0.4650879 ] 3   4 
[-1.7333984   0.05032349  0.01600647  0.6147461   0.5761719   0.39233398] 3   1 
[-0.9370117   0.26391602 -0.00733566  0.21606445  0.24243164  0.05212402] 1   1 Match 67

[-1.65625     0.01708984 -0.4230957   0.38916016  0.8618164   0.8515625 ] 4   1 
[-0.08459473  0.640625    0.17797852 -0.11303711 -0.3540039  -0.5625    ] 1   0 
[-0.50634766  0.38012695  0.46264648  0.4807129  -0.07983398 -0.60009766] 3   4 
[-0.22229004  0.3540039   0.41381836  0.2775879  -0.2241211  -0.50683594] 2   1 
[-1.214843

[-1.0644531   0.5053711  -0.3059082   0.13244629  0.39282227  0.08886719] 1   1 Match 104

[-0.20019531  0.5527344   0.34570312  0.25683594 -0.4453125  -0.71484375] 1   4 
[-1.5800781   0.06671143 -0.22058105  0.46948242  0.6879883   0.43554688] 4   3 
[-0.8486328   0.5678711  -0.05950928  0.19555664  0.02639771  0.04849243] 1   3 
[-1.6318359  -0.07629395 -0.16784668  0.75878906  0.6855469   0.6069336 ] 3   3 Match 105

[-1.7910156  -0.20397949 -0.29882812  0.7421875   0.8647461   0.7211914 ] 4   3 
[ 0.13659668  0.42578125  0.44628906  0.0914917  -0.4868164  -0.65722656] 2   2 Match 106

[-0.20935059  0.44702148  0.5336914   0.30737305 -0.18469238 -0.77685547] 2   1 
[-1.46875     0.2565918  -0.32641602  0.51171875  0.47509766  0.39575195] 3   3 Match 107

[-0.48754883  0.35498047  0.19848633  0.26538086  0.18530273 -0.12634277] 1   3 
[ 0.19592285  0.52490234  0.60302734  0.24621582 -0.54296875 -0.9741211 ] 2   1 
[-1.2480469   1.7294922  -0.8828125   0.6088867  -0.7480469  -0.13757

[-0.5180664   0.26342773  0.27368164  0.4814453  -0.14367676 -0.43310547] 3   3 Match 146

[-1.6347656  -0.10821533 -0.46240234  0.30615234  1.0761719   0.8754883 ] 4   5 
[ 0.35742188  0.58447266  0.6044922   0.05612183 -0.68652344 -0.98828125] 2   1 
[-1.515625    0.23791504 -0.07592773  0.5600586   0.57470703  0.35888672] 4   1 
[-1.2177734   0.05496216 -0.46875     0.02307129  0.75878906  1.0429688 ] 5   4 
[-1.6679688  -0.07183838 -0.03833008  0.71728516  0.8046875   0.29174805] 4   5 
[-1.2861328   0.38671875 -0.06518555  0.3798828   0.32641602  0.34570312] 1   3 
[-0.8725586   0.00682068  0.49243164  0.90625     0.17407227 -0.30932617] 3   5 
[ 0.39038086  0.59814453  0.42626953 -0.16967773 -0.65527344 -0.75927734] 1   1 Match 147

[-1.8837891  -0.17382812 -0.40600586  0.6748047   0.87158203  0.75683594] 4   5 
[-1.2001953   0.16394043  0.20471191  0.9370117   0.44873047  0.09191895] 3   1 
[ 0.16333008  0.6269531   0.41381836 -0.02523804 -0.46191406 -0.8457031 ] 1   3 
[-1.0732

[-1.6552734   0.0397644   0.02697754  0.8334961   0.49780273  0.12939453] 3   2 
[-0.36572266  0.39624023  0.32202148  0.39868164 -0.19799805 -0.55371094] 3   0 
[-0.7783203   0.4921875   0.29907227  0.39672852 -0.00110149 -0.47070312] 1   2 
[-0.00092411  0.78222656  0.17053223 -0.11157227 -0.31298828 -0.5727539 ] 1   2 
[-0.8701172   0.05743408  0.45239258  0.76171875  0.13098145 -0.2614746 ] 3   1 
[-1.2675781   0.2705078  -0.22741699  0.29785156  0.45947266  0.6357422 ] 5   4 
[-0.5366211   0.43701172  0.33544922  0.26831055 -0.14257812 -0.4416504 ] 1   2 
[-1.28125     0.03335571 -0.05233765  0.5419922   0.61865234  0.6323242 ] 5   3 
[-0.02659607  0.5966797   0.16186523 -0.08587646 -0.3930664  -0.4104004 ] 1   3 
[-1.9257812  -0.09655762 -0.18164062  0.56640625  0.9692383   0.5996094 ] 4   4 Match 178

[-1.3466797   0.22338867  0.27197266  0.85839844  0.3527832  -0.07745361] 3   2 
[-1.0107422  -0.05831909  0.4501953   0.75146484  0.33642578 -0.22851562] 3   2 
[-1.7695312  -0.10

[-0.7832031   0.35009766 -0.03475952  0.25585938  0.29296875  0.18835449] 1   4 
[ 0.4326172   0.7060547   0.5566406  -0.00597    -0.76660156 -0.9736328 ] 1   0 
[-1.4580078   0.105896   -0.4892578   0.2578125   0.90527344  1.1962891 ] 5   2 
[-1.2597656   0.32226562 -0.23535156  0.43701172  0.58935547  0.17248535] 4   1 
[-1.1748047   0.35058594 -0.51220703  0.0947876   0.4873047   0.9980469 ] 5   5 Match 214

[-1.4365234   0.04425049  0.11975098  0.8203125   0.53808594  0.04959106] 3   2 
[-1.7490234  -0.08123779 -0.43725586  0.27539062  1.0322266   0.8535156 ] 4   4 Match 215

[-1.5166016  -0.10943604  0.0519104   0.8305664   0.63671875  0.43701172] 3   3 Match 216

[-0.61279297  0.29516602  0.20776367  0.4482422  -0.0569458  -0.3232422 ] 3   4 
[ 0.24853516  0.61279297  0.44067383  0.0657959  -0.55029297 -0.8227539 ] 1   5 
[-0.7783203   0.44677734  0.32885742  0.4819336   0.12634277 -0.19506836] 3   1 
[ 0.30688477  0.81103516  0.36132812 -0.08197021 -0.7597656  -0.8745117 ] 1   0

[-0.24169922  0.51904297  0.18005371  0.1315918  -0.19140625 -0.47998047] 1   0 
[ 0.42773438  0.7734375   0.39697266 -0.05297852 -0.5751953  -0.98339844] 1   2 
[ 0.23571777  0.52978516  0.5366211   0.13354492 -0.5517578  -0.9897461 ] 2   1 
[-1.5712891  -0.24291992 -0.36645508  0.28125     1.1113281   0.83203125] 4   4 Match 250

[-0.9316406   0.30151367 -0.04537964  0.32836914  0.28442383 -0.00846863] 3   1 
[-0.67333984  0.46679688  0.23632812  0.27856445  0.10498047 -0.12976074] 1   3 
[-0.23999023  0.5722656   0.09210205  0.1307373  -0.03662109 -0.2998047 ] 1   0 
[-0.9316406   0.27954102 -0.47802734 -0.07141113  0.5522461   1.1455078 ] 5   1 
[-1.3798828   0.35717773  0.05773926  0.40356445  0.48754883  0.29003906] 4   3 
[-1.7744141  -0.08203125 -0.3876953   0.4501953   0.890625    0.7939453 ] 4   5 
[-1.0078125   0.27075195  0.23754883  0.49853516  0.28588867 -0.07904053] 3   4 
[-1.6162109  -0.09893799 -0.23986816  0.43969727  0.8930664   0.5888672 ] 4   4 Match 251

[-0.1007

[-0.8017578   0.41967773 -0.30664062  0.03198242  0.3503418   0.26953125] 1   5 
[-1.4726562  -0.00732422 -0.00592804  0.7631836   0.47387695  0.23730469] 3   1 
[-1.1464844   0.4140625  -0.33740234  0.10565186  0.5878906   0.5493164 ] 4   4 Match 276

[-1.6884766  -0.21130371 -0.39624023  0.53027344  0.9892578   0.9423828 ] 4   2 
[ 0.05853271  0.52783203  0.2397461  -0.05172729 -0.50097656 -0.43554688] 1   5 
[-0.15661621  0.67871094  0.20739746  0.02893066 -0.35961914 -0.61328125] 1   1 Match 277

[-1.1025391   0.09515381 -0.3774414   0.11206055  0.68896484  0.86328125] 5   5 Match 278

[-1.5859375  -0.15234375 -0.48413086  0.2631836   0.97021484  1.1015625 ] 5   4 
[ 0.12512207  0.63671875  0.28637695  0.02590942 -0.4790039  -0.7246094 ] 1   5 
[-0.9838867   0.49169922 -0.00822449  0.17687988  0.19970703  0.19506836] 1   4 
[-0.07757568  0.52441406  0.3215332   0.09924316 -0.16491699 -0.39111328] 1   1 Match 279

[-0.13708496  0.4267578   0.5253906   0.25805664 -0.39892578 -0.79785

[-0.70751953  0.3713379   0.15466309  0.34765625 -0.00589752 -0.04501343] 1   2 
[ 0.24743652  0.69970703  0.3112793  -0.09326172 -0.5419922  -0.7788086 ] 1   5 
[-1.6884766   0.11071777 -0.29785156  0.25708008  0.75        0.94921875] 5   2 
[-1.2685547  -0.0690918  -0.01264954  0.73779297  0.5839844  -0.02339172] 3   2 
[-0.6254883   0.5854492  -0.13916016  0.08776855  0.21960449  0.00816345] 1   1 Match 315

[-1.4316406   0.01082611  0.04742432  0.74853516  0.53027344 -0.00338554] 3   5 
[-0.07781982  0.49780273  0.32250977  0.07177734 -0.34594727 -0.5546875 ] 1   4 
[-0.16955566  0.5908203   0.04888916  0.04660034 -0.2286377  -0.48413086] 1   1 Match 316

[-0.71533203  0.48999023 -0.28100586  0.09075928  0.31347656  0.50927734] 5   2 
[-1.0185547   0.25048828 -0.09136963  0.44970703  0.2006836   0.2758789 ] 3   3 Match 317

[ 0.24682617  0.71435547  0.02806091  0.3544922  -0.3137207  -0.39501953] 1   1 Match 318

[-0.5654297   0.43481445 -0.26000977  0.09216309  0.23535156  0.36816

In [63]:
from sklearn import metrics
 
print(metrics.confusion_matrix(test['labels'],Pred))

[[ 0 49 13 14 13  3]
 [ 0 99 25 55 42 29]
 [ 0 74 25 52 45 18]
 [ 0 88 14 84 62 19]
 [ 0 46  9 70 82 42]
 [ 0 41  3 48 72 47]]


In [64]:
target_names = ['Pants', 'False', 'Barely-True','Half-True','Mostly-True','True']

print(metrics.classification_report(test['labels'], Pred,target_names =target_names))

              precision    recall  f1-score   support

       Pants       0.00      0.00      0.00        92
       False       0.25      0.40      0.31       250
 Barely-True       0.28      0.12      0.17       214
   Half-True       0.26      0.31      0.28       267
 Mostly-True       0.26      0.33      0.29       249
        True       0.30      0.22      0.25       211

    accuracy                           0.26      1283
   macro avg       0.22      0.23      0.22      1283
weighted avg       0.25      0.26      0.24      1283



In [65]:
Fold_Predictions=pd.DataFrame(Pred, columns=['Pred2'] )
Fold_Predictions

,Pred2
0,4
1,1
2,1
3,3
4,4
...,...
1278,4
1279,1
1280,1
1281,1


In [68]:
Fold_Predictions.to_excel(output_folder+'/Saves/fold2_Predictions.xls')

In [69]:
#clearing GPU cache

del(model)
del(TrainResult, TrainModel_outputs, EvalResult, EvalModel_outputs, TestResult, TestModel_outputs, wrong_predictions)
torch.cuda.empty_cache()

NameError: name 'model' is not defined

## Fold 3: training & caturing predictions

In [70]:
fold_number='3'

train=pd.read_excel('./folds/train_fold'+fold_number+'.xls')
Eval=pd.read_excel('./folds/valid'+fold_number+'.xls') #evaluation set


In [87]:
 
output_folder='./folds/fold'+fold_number+'/'+model_class+'/'+model_version+"/"
cache_directory= "./folds/fold"+fold_number+'/'+model_class+"/"+model_version+"/cache/"


print('model variables were set up: ')

 
save_every_steps=1285
# assuming training batch size of 8
# any number above 1284 saves the model only at every epoch
# Saving the model mid training very often will consume disk space fast

train_args={
    "output_dir":output_folder,
    "cache_dir":cache_directory,
    'reprocess_input_data': True,
    'overwrite_output_dir': True,
    'num_train_epochs': 1,
    "save_steps": save_every_steps, 
    "learning_rate":0.9e-5,
    "train_batch_size": 64,
    "eval_batch_size": 16,
    "evaluate_during_training_steps": 5,
    "max_seq_length": 100,
    "n_gpu": 1,
}

# Create a ClassificationModel
model = ClassificationModel(model_class, model_version, num_labels=labels_count, args=train_args) 

model variables were set up: 


In [88]:
# loading the checkpoint that gave the best result
 
CheckPoint='checkpoint-145-epoch-3'  

preSavedCheckpoint=output_folder+CheckPoint

print('Loading model, please wait...')
model = ClassificationModel( model_class, preSavedCheckpoint, num_labels=labels_count, args=train_args) 
print('model in use is :', preSavedCheckpoint )
 

Loading model, please wait...
model in use is : ./folds/fold3/bert/bert-base-cased/checkpoint-145-epoch-3


In [89]:
# Train the model
current_time = datetime.now()
model.train_model(train)
print("Training time: ", datetime.now() - current_time)

Converting to features started. Cache is not used.



Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic


Running loss: 1.731089

Training of bert model complete. Saved to ./folds/fold3/bert/bert-base-cased/.
Training time:  0:03:39.424025


In [90]:
TrainResult, TrainModel_outputs, wrong_predictions = model.eval_model(train, acc=sklearn.metrics.accuracy_score)
 
EvalResult, EvalModel_outputs, wrong_predictions = model.eval_model(Eval, acc=sklearn.metrics.accuracy_score)

TestResult, TestModel_outputs, wrong_predictions = model.eval_model(test, acc=sklearn.metrics.accuracy_score)

print('Training Result:', TrainResult['acc'])
#print('Model Out:', TrainModel_outputs)

print('Eval Result:', EvalResult['acc'])
#print('Model Out:', EvalModel_outputs)

print('Test Set Result:', TestResult['acc'])
#print('Model Out:', TestModel_outputs)

Features loaded from cache at ./folds/fold3/bert/bert-base-cased/cache/cached_dev_bert_100_6_9245



{'mcc': 0.16043308795842504, 'acc': 0.3145484045429962, 'eval_loss': 1.6065435729224788}
Features loaded from cache at ./folds/fold3/bert/bert-base-cased/cache/cached_dev_bert_100_6_2308



{'mcc': 0.09241782166110475, 'acc': 0.2634315424610052, 'eval_loss': 1.6902727932765567}
Features loaded from cache at ./folds/fold3/bert/bert-base-cased/cache/cached_dev_bert_100_6_1283



{'mcc': 0.08667648410591959, 'acc': 0.25954793452844893, 'eval_loss': 1.6734194034411582}
Training Result: 0.3145484045429962
Eval Result: 0.2634315424610052
Test Set Result: 0.25954793452844893


In [91]:
Pred=[]

countCorrect=0

for row in range(TestModel_outputs.shape[0]):
    outputs=TestModel_outputs[row]
    #print(test.iloc[row,0])
    print(outputs, end=' ')
    
    result=0
    if outputs[0]<outputs[1]:result=1
    if outputs[result]<outputs[2]:result=2
    if outputs[result]<outputs[3]:result=3
    if outputs[result]<outputs[4]:result=4
    if outputs[result]<outputs[5]:result=5
    Pred.append(result)
    print(result, ' ',test.iloc[row,1], end=' ')
    if result==test.iloc[row,1]:
        countCorrect+=1
        print('Match',countCorrect)
    print('')

print(countCorrect)
#Pred

[-1.8212891  -0.02378845 -0.37304688  0.38549805  1.0205078   0.93408203] 4   4 Match 1

[-0.39233398  0.7783203   0.52978516  0.5541992   0.2244873   0.0219574 ] 1   3 
[-0.9135742   0.52734375  0.25        0.6269531   0.5571289   0.48046875] 3   1 
[-6.0742188e-01  4.1699219e-01  3.5961914e-01  7.3681641e-01
  3.1787109e-01  3.5953522e-04] 3   5 
[-1.8847656  -0.38671875 -0.22583008  0.69384766  1.0087891   0.6489258 ] 4   5 
[-1.4150391   0.11456299 -0.13049316  0.7861328   1.0361328   0.85498047] 4   2 
[-2.0410156  -0.21044922 -0.18640137  0.59521484  1.1171875   0.796875  ] 4   4 Match 2

[-0.6582031   0.15393066  0.20361328  0.55908203  0.21142578 -0.05899048] 3   5 
[-1.7822266   0.13842773 -0.38549805  0.2010498   0.8964844   0.99365234] 5   4 
[-0.66503906  0.29467773  0.2401123   0.56396484  0.24768066 -0.08355713] 3   5 
[ 0.12145996  0.78125     0.546875    0.46069336 -0.16931152 -0.53027344] 1   5 
[-0.7265625   0.41455078  0.31591797  0.63427734  0.2939453   0.06573486] 

[-1.7558594  -0.03793335 -0.11987305  0.57470703  1.0039062   0.8979492 ] 4   5 
[-1.9707031  -0.23962402 -0.08959961  0.6591797   1.140625    0.7890625 ] 4   4 Match 27

[-0.32250977  1.1376953   0.4963379   0.5654297   0.0304718   0.14648438] 1   1 Match 28

[ 0.31689453  0.5654297   0.5         0.28881836 -0.3449707  -0.7158203 ] 1   0 
[-0.38549805  0.78564453  0.41601562  0.34692383  0.10101318 -0.04400635] 1   5 
[-0.7158203   0.08990479  0.27124023  0.7426758   0.30786133 -0.09564209] 3   1 
[ 0.19482422  0.5810547   0.58251953  0.45141602 -0.2607422  -0.6152344 ] 2   1 
[-1.8730469  -0.42089844 -0.04876709  0.76953125  1.03125     0.6557617 ] 4   3 
[-0.6982422   0.375       0.3046875   0.5546875   0.24316406  0.01082611] 3   2 
[-0.3552246   0.80615234  0.5527344   0.5493164   0.12097168 -0.0249176 ] 1   2 
[-1.5722656   0.34985352 -0.4086914   0.18261719  0.8955078   0.9067383 ] 5   4 
[-1.5732422   0.18237305 -0.11804199  0.4597168   0.9707031   0.80908203] 4   4 Match 29

[

[ 0.23669434  0.90185547  0.60498047  0.42700195 -0.27441406 -0.52734375] 1   3 
[ 0.24719238  0.3310547   0.42358398  0.31713867 -0.34521484 -0.67822266] 2   5 
[ 0.40820312  0.36987305  0.4189453   0.23425293 -0.41430664 -0.67529297] 2   0 
[-2.03125    -0.20861816 -0.36083984  0.40014648  1.          0.91503906] 4   4 Match 56

[ 0.24853516  0.36376953  0.56103516  0.37817383 -0.29467773 -0.73876953] 2   2 Match 57

[ 0.38183594  0.30932617  0.36938477  0.24597168 -0.44360352 -0.7133789 ] 0   2 
[-0.14819336  0.8071289   0.46240234  0.31640625 -0.11279297 -0.24523926] 1   1 Match 58

[ 0.05645752  0.40625     0.5239258   0.4501953  -0.19677734 -0.60058594] 2   3 
[-0.11114502  0.9433594   0.59716797  0.46166992  0.02210999 -0.07781982] 1   5 
[ 0.3100586   0.62841797  0.5419922   0.2836914  -0.35498047 -0.6503906 ] 1   4 
[-0.0193634   0.5004883   0.5097656   0.41479492 -0.16748047 -0.5131836 ] 2   1 
[ 0.5307617   0.5576172   0.56640625  0.32202148 -0.39819336 -0.75390625] 2   0 
[

[-0.2631836   0.7841797   0.4975586   0.4519043   0.0617981  -0.08428955] 1   2 
[-1.9326172 -0.0836792 -0.3642578  0.2836914  0.9609375  0.8876953] 4   4 Match 96

[-1.0058594  -0.09735107  0.08154297  0.68847656  0.42773438  0.0871582 ] 3   1 
[-0.55029297  0.74902344  0.40478516  0.5888672   0.42358398  0.1586914 ] 1   1 Match 97

[-0.06561279  1.2675781   0.5449219   0.5698242  -0.09130859  0.06445312] 1   1 Match 98

[-0.3779297   0.30517578  0.30249023  0.43115234 -0.00938416 -0.2487793 ] 3   2 
[-1.4335938   0.3395996  -0.3605957   0.26293945  0.88427734  0.89160156] 5   0 
[ 0.24987793  0.6772461   0.5341797   0.31030273 -0.29052734 -0.5595703 ] 1   4 
[ 0.20031738  0.79833984  0.54833984  0.32299805 -0.27441406 -0.52685547] 1   1 Match 99

[-2.1074219  -0.4465332  -0.16467285  0.67041016  1.125       0.7597656 ] 4   4 Match 100

[-0.19567871  0.3701172   0.28637695  0.32543945 -0.1694336  -0.3400879 ] 1   1 Match 101

[-0.8339844   0.5371094   0.25830078  0.50390625  0.4038086

[-0.8798828   0.06027222  0.26831055  0.87109375  0.43286133  0.15222168] 3   2 
[-1.5898438   0.07891846 -0.2890625   0.39648438  0.92529297  0.8901367 ] 4   4 Match 135

[-0.3137207   0.4375      0.48510742  0.5332031   0.03689575 -0.3647461 ] 3   2 
[ 0.19934082  0.94873047  0.57421875  0.41577148 -0.2824707  -0.47851562] 1   2 
[-0.68408203  0.8769531   0.42822266  0.50341797  0.35253906  0.41723633] 1   3 
[-1.0800781   0.60791016  0.00567245  0.5546875   0.7529297   0.8022461 ] 5   5 Match 136

[ 0.28564453  0.9350586   0.5810547   0.36645508 -0.3227539  -0.4892578 ] 1   2 
[-1.390625   -0.2998047   0.05734253  0.8364258   0.71484375  0.2836914 ] 3   0 
[-2.0078125  -0.31835938 -0.20812988  0.76904297  1.1611328   0.8588867 ] 4   4 Match 137

[ 0.21313477  0.77783203  0.58154297  0.44360352 -0.21606445 -0.6464844 ] 1   1 Match 138

[ 0.4814453   0.58496094  0.5107422   0.29418945 -0.39331055 -0.72509766] 1   1 Match 139

[ 0.5605469   0.53564453  0.453125    0.27856445 -0.4299316

...

[-0.07659912  0.92041016  0.54589844  0.40185547 -0.10809326 -0.23327637] 1   4 
[-1.1591797   0.62939453 -0.04644775  0.39233398  0.7314453   0.7373047 ] 5   4 
[-0.3071289   0.31689453  0.3305664   0.46899414 -0.04840088 -0.3413086 ] 3   3 Match 195

[-2.0351562  -0.4453125  -0.12878418  0.7939453   1.1035156   0.7241211 ] 4   3 
[-0.9824219   0.21276855  0.15637207  0.5932617   0.41967773  0.22607422] 3   2 
[ 0.34033203  0.7182617   0.5463867   0.32055664 -0.2919922  -0.6308594 ] 1   0 
[ 0.14331055  0.6323242   0.45629883  0.28198242 -0.27807617 -0.50439453] 1   1 Match 196

[-1.9199219  -0.14331055 -0.30126953  0.42041016  0.97021484  0.8261719 ] 4   4 Match 197

[-1.8369141  -0.28076172 -0.05422974  0.8876953   1.1435547   0.7685547 ] 4   4 Match 198

[-2.0039062  -0.2919922  -0.21350098  0.6381836   1.1396484   0.859375  ] 4   4 Match 199

[-1.9316406  -0.17590332 -0.16247559  0.5957031   1.0449219   0.82470703] 4   5 
[-0.7128906  -0.22314453  0.04959106  0.6171875   0.1580810

[ 0.22363281  0.47753906  0.38354492  0.27270508 -0.3256836  -0.57958984] 1   1 Match 230

[-0.6557617   0.75341797  0.34765625  0.5463867   0.4555664   0.5209961 ] 1   2 
[-1.9345703  -0.38232422 -0.18518066  0.69677734  0.9628906   0.66748047] 4   4 Match 231

[ 0.23217773  0.6333008   0.40771484  0.15551758 -0.31420898 -0.50634766] 1   5 
[ 0.3466797   0.859375    0.5673828   0.36157227 -0.3449707  -0.6435547 ] 1   5 
[-1.3935547  -0.06335449  0.00811768  0.7661133   0.73583984  0.46972656] 3   1 
[-1.1210938  -0.3034668  -0.04107666  0.69677734  0.41845703  0.05923462] 3   3 Match 232

[-1.6689453   0.10272217 -0.23156738  0.44702148  1.0234375   0.84228516] 4   1 
[-0.18115234  1.0791016   0.61621094  0.50439453  0.09753418  0.18652344] 1   2 
[-1.6796875  -0.11846924  0.00878906  0.67871094  1.0029297   0.68408203] 4   3 
[-2.0449219  -0.39111328 -0.16882324  0.69921875  1.140625    0.8251953 ] 4   1 
[ 0.36767578  0.4765625   0.49414062  0.28076172 -0.35375977 -0.6611328 ] 2   0

[-1.03125     0.58740234  0.20141602  0.5385742   0.5600586   0.4909668 ] 1   1 Match 262

[-0.39770508  0.89941406  0.49023438  0.5097656   0.18371582  0.01655579] 1   4 
[-1.2275391   0.59765625 -0.15673828  0.38500977  0.8334961   0.81103516] 4   4 Match 263

[-1.3964844   0.02320862  0.03973389  0.6323242   0.80126953  0.54833984] 4   3 
[-1.8476562  -0.38671875 -0.0440979   0.80078125  0.9448242   0.5991211 ] 4   3 
[-0.07196045  0.6567383   0.5253906   0.42797852 -0.08032227 -0.42333984] 1   3 
[-0.27856445  0.67041016  0.44384766  0.43017578  0.02441406 -0.18579102] 1   2 
[-1.9257812  -0.23474121 -0.29125977  0.5415039   1.0488281   0.82470703] 4   3 
[-1.6445312  -0.2064209  -0.07318115  0.6459961   0.8388672   0.6245117 ] 4   3 
[ 0.37841797  0.81591797  0.5649414   0.3630371  -0.33154297 -0.6357422 ] 1   4 
[-1.6132812  -0.20422363  0.00508881  0.75439453  0.9267578   0.5830078 ] 4   1 
[-1.7998047   0.03289795 -0.39233398  0.24829102  0.9458008   0.96777344] 5   4 
[-2.0507

[-0.37890625  0.8857422   0.52734375  0.5786133   0.2553711   0.0413208 ] 1   4 
[-1.9541016  -0.25854492 -0.2166748   0.60302734  1.0576172   0.8847656 ] 4   5 
[-0.84277344  0.11901855  0.17211914  0.63720703  0.28710938  0.03726196] 3   3 Match 293

[ 0.01579285  0.65478516  0.4230957   0.29077148 -0.18688965 -0.4086914 ] 1   0 
[ 0.12030029  0.34423828  0.4189453   0.3100586  -0.2939453  -0.61328125] 2   2 Match 294

[-0.5258789   0.63134766  0.48999023  0.6582031   0.2915039   0.07214355] 3   1 
[-0.9199219   0.14501953  0.21398926  0.75927734  0.4555664   0.14025879] 3   2 
[ 0.28881836  0.3798828   0.4440918   0.23803711 -0.3671875  -0.65722656] 2   1 
[-1.4550781   0.01478577 -0.02139282  0.65527344  0.81152344  0.6230469 ] 4   5 
[-0.07574463  0.24365234  0.36010742  0.4267578  -0.17053223 -0.49365234] 3   2 
[-0.10858154  0.58154297  0.44335938  0.3762207  -0.16699219 -0.40307617] 1   0 
[-1.0546875  0.6201172  0.1159668  0.5708008  0.6542969  0.6191406] 4   2 
[ 0.21887207  

[-0.23901367  0.5644531   0.4711914   0.46557617  0.00150108 -0.27856445] 1   2 
[-0.03668213  0.52783203  0.52001953  0.46289062 -0.11987305 -0.5263672 ] 1   5 
[ 0.17651367  0.3371582   0.46801758  0.35180664 -0.2734375  -0.65722656] 2   5 
[-0.9345703   0.7006836   0.26098633  0.60058594  0.5776367   0.50927734] 1   3 
[ 0.0748291   0.6357422   0.53125     0.37646484 -0.22167969 -0.5385742 ] 1   3 
[ 0.08886719  0.76660156  0.5361328   0.3269043  -0.20361328 -0.46850586] 1   2 
[-1.8535156  -0.12188721 -0.23510742  0.5864258   1.0703125   0.8491211 ] 4   4 Match 325

[-0.5800781   0.29760742  0.37597656  0.68310547  0.22814941 -0.16625977] 3   4 
[-0.95703125  0.45117188  0.2705078   0.6479492   0.6298828   0.2956543 ] 3   3 Match 326

[ 0.18151855  0.8022461   0.59472656  0.46777344 -0.19311523 -0.5986328 ] 1   1 Match 327

[-0.0045166   0.46264648  0.4807129   0.46069336 -0.16333008 -0.5439453 ] 2   3 
[-0.38842773  0.7675781   0.5595703   0.6743164   0.24768066 -0.11157227] 1   4

In [92]:
from sklearn import metrics
 
print(metrics.confusion_matrix(test['labels'],Pred))

[[  3  50  13  11  13   2]
 [  2 125  17  46  52   8]
 [  3  92  22  40  47  10]
 [  1  94  17  58  91   6]
 [  0  61   4  58 111  15]
 [  0  50   6  30 111  14]]


In [93]:
target_names = ['Pants', 'False', 'Barely-True','Half-True','Mostly-True','True']

print(metrics.classification_report(test['labels'], Pred,target_names =target_names))

              precision    recall  f1-score   support

       Pants       0.33      0.03      0.06        92
       False       0.26      0.50      0.35       250
 Barely-True       0.28      0.10      0.15       214
   Half-True       0.24      0.22      0.23       267
 Mostly-True       0.26      0.45      0.33       249
        True       0.25      0.07      0.11       211

    accuracy                           0.26      1283
   macro avg       0.27      0.23      0.20      1283
weighted avg       0.26      0.26      0.23      1283



In [94]:
Fold_Predictions=pd.DataFrame(Pred, columns=['Pred3'] )
Fold_Predictions

,Pred3
0,4
1,1
2,3
3,3
4,4
...,...
1278,4
1279,3
1280,0
1281,2


In [95]:
Fold_Predictions.to_excel(output_folder+'/Saves/fold3_Predictions.xls')

In [96]:
#clearing GPU cache

del(model)
del(TrainResult, TrainModel_outputs, EvalResult, EvalModel_outputs, TestResult, TestModel_outputs, wrong_predictions)
torch.cuda.empty_cache()

## Fold 4: training & capturing predictions

In [97]:
fold_number='4'

train=pd.read_excel('./folds/train_fold'+fold_number+'.xls')
Eval=pd.read_excel('./folds/valid'+fold_number+'.xls') #evaluation set


In [113]:

output_folder='./folds/fold'+fold_number+'/'+model_class+'/'+model_version+"/"
cache_directory= "./folds/fold"+fold_number+'/'+model_class+"/"+model_version+"/cache/"


print('model variables were set up: ')

 
save_every_steps=1285
# assuming training batch size of 8
# any number above 1284 saves the model only at every epoch
# Saving the model mid training very often will consume disk space fast

train_args={
    "output_dir":output_folder,
    "cache_dir":cache_directory,
    'reprocess_input_data': True,
    'overwrite_output_dir': True,
    'num_train_epochs': 1,
    "save_steps": save_every_steps, 
    "learning_rate": 0.9e-5,
    "train_batch_size": 64,
    "eval_batch_size": 16,
    "adam_epsilon": 1e-8,
    "evaluate_during_training_steps": 5,
    "max_seq_length": 120,
    "n_gpu": 1,
}

# Create a ClassificationModel
model = ClassificationModel(model_class, model_version, num_labels=labels_count, args=train_args) 

model variables were set up: 


In [114]:
# loading the checkpoint that gave the best result

CheckPoint='checkpoint-145-epoch-1' 


preSavedCheckpoint=output_folder+CheckPoint

print('Loading model, please wait...')
model = ClassificationModel( model_class, preSavedCheckpoint, num_labels=labels_count, args=train_args) 
print('model in use is :', preSavedCheckpoint )


Loading model, please wait...
model in use is : ./folds/fold4/bert/bert-base-cased/checkpoint-145-epoch-1


In [115]:
# Train the model
current_time = datetime.now()
model.train_model(train)
print("Training time: ", datetime.now() - current_time, 'at: ' ,datetime.now())

Converting to features started. Cache is not used.



Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic


Running loss: 1.555958

Training of bert model complete. Saved to ./folds/fold4/bert/bert-base-cased/.
Training time:  0:04:01.393322 at:  2020-04-29 14:49:16.610980


In [116]:
TrainResult, TrainModel_outputs, wrong_predictions = model.eval_model(train, acc=sklearn.metrics.accuracy_score)
 
EvalResult, EvalModel_outputs, wrong_predictions = model.eval_model(Eval, acc=sklearn.metrics.accuracy_score)

TestResult, TestModel_outputs, wrong_predictions = model.eval_model(test, acc=sklearn.metrics.accuracy_score)

print('Training Result:', TrainResult['acc'])
#print('Model Out:', TrainModel_outputs)

print('Eval Result:', EvalResult['acc'])
#print('Model Out:', EvalModel_outputs)

print('Test Set Result:', TestResult['acc'])
#print('Model Out:', TestModel_outputs)

Features loaded from cache at ./folds/fold4/bert/bert-base-cased/cache/cached_dev_bert_120_6_9245



{'mcc': 0.2289947611707765, 'acc': 0.37155219037317466, 'eval_loss': 1.5386967634247248}
Features loaded from cache at ./folds/fold4/bert/bert-base-cased/cache/cached_dev_bert_120_6_2308



{'mcc': 0.08055470028259404, 'acc': 0.25433275563258234, 'eval_loss': 1.7059699535369872}
Features loaded from cache at ./folds/fold4/bert/bert-base-cased/cache/cached_dev_bert_120_6_1283



{'mcc': 0.10783833267909908, 'acc': 0.2774746687451286, 'eval_loss': 1.6666809761965717}
Training Result: 0.37155219037317466
Eval Result: 0.25433275563258234
Test Set Result: 0.2774746687451286


In [117]:
Pred=[]

countCorrect=0

for row in range(TestModel_outputs.shape[0]):
    outputs=TestModel_outputs[row]
    #print(test.iloc[row,0])
    print(outputs, end=' ')
    
    result=0
    if outputs[0]<outputs[1]:result=1
    if outputs[result]<outputs[2]:result=2
    if outputs[result]<outputs[3]:result=3
    if outputs[result]<outputs[4]:result=4
    if outputs[result]<outputs[5]:result=5
    Pred.append(result)
    print(result, ' ',test.iloc[row,1], end=' ')
    if result==test.iloc[row,1]:
        countCorrect+=1
        print('Match',countCorrect)
    print('')

print(countCorrect)
#Pred

[-1.7568359  -0.07763672 -0.36523438  0.45385742  1.1386719   0.83154297] 4   4 Match 1

[-0.17553711  0.42456055  0.4519043   0.17016602 -0.5371094  -1.0380859 ] 2   3 
[-0.02114868  0.35351562  0.24560547  0.08374023 -0.29492188 -0.50439453] 1   1 Match 2

[-0.8261719   0.03207397  0.22827148  0.59472656 -0.00725174 -0.45458984] 3   5 
[-1.7207031  -0.19433594 -0.5566406   0.7167969   1.1601562   0.58496094] 4   5 
[-0.6621094   0.27612305  0.11517334  0.23120117 -0.09674072 -0.19006348] 1   2 
[-1.6962891  -0.49072266 -0.40625     0.8183594   1.1914062   0.3840332 ] 4   4 Match 3

[-0.8588867   0.3864746   0.0668335   0.40307617  0.00743866 -0.51220703] 3   5 
[-1.3710938  -0.22839355 -0.04525757 -0.09753418  0.71484375  0.89501953] 5   4 
[-0.9428711   0.23583984 -0.1050415   0.6640625   0.09136963 -0.62646484] 3   5 
[-0.40844727  0.13793945  0.3635254   0.46557617 -0.38183594 -1.109375  ] 3   5 
[-0.30126953  0.41479492  0.26000977  0.2220459  -0.421875   -0.53564453] 1   3 
[-0.

[-1.6542969   0.2331543  -0.11975098  0.40771484  0.5859375   0.5263672 ] 4   2 
[-0.04171753  0.5961914   0.26757812  0.10858154 -0.5185547  -0.8544922 ] 1   0 
[-1.4873047  -0.24230957  0.11724854  0.7841797   0.5517578   0.06652832] 3   2 
[-1.4931641  -0.19140625  0.10107422  0.36694336  0.45263672  0.35668945] 4   5 
[-1.7978516  -0.3503418  -0.17700195  0.8359375   0.9716797   0.15100098] 4   4 Match 27

[-0.74902344  1.8769531  -0.7348633   0.39135742 -0.89160156 -0.73876953] 1   1 Match 28

[ 0.0073204   0.4038086   0.6191406   0.1430664  -0.55615234 -1.1503906 ] 2   0 
[-0.2927246   0.90625     0.10723877  0.17370605 -0.30297852 -0.4802246 ] 1   5 
[-1.7802734  -0.15563965  0.04766846  0.9145508   0.6113281   0.13989258] 3   1 
[-0.05349731  0.4741211   0.5830078   0.3491211  -0.66064453 -1.1171875 ] 2   1 
[-1.7226562  -0.36206055 -0.4260254   0.77197266  1.1503906   0.58154297] 4   3 
[-0.4975586   0.53515625  0.2626953   0.33154297 -0.38183594 -0.7036133 ] 1   2 
[-0.075500

[-0.9477539   0.18518066 -0.08087158  0.4506836   0.14050293 -0.13061523] 3   1 
[ 0.4892578   0.6088867   0.41723633  0.2775879  -0.9243164  -1.2353516 ] 1   0 
[-0.8413086   0.36206055 -0.12585449 -0.05944824  0.25195312  0.5390625 ] 5   0 
[-1.4892578  -0.03286743 -0.15917969  0.8984375   0.2631836  -0.10101318] 3   1 
[-0.28808594  0.27246094  0.10644531  0.10803223 -0.16577148 -0.59033203] 1   3 
[-0.62939453  0.1694336   0.43066406  0.3972168  -0.37939453 -0.56591797] 2   5 
[ 0.5258789   0.5776367   0.30444336 -0.23120117 -0.7192383  -0.7988281 ] 1   0 
[-1.7402344  -0.24951172 -0.35864258  0.45654297  0.9897461   0.95751953] 4   4 Match 58

[-0.56347656  0.05130005  0.7001953   0.6464844  -0.3671875  -1.1591797 ] 2   2 Match 59

[ 0.5419922   0.5678711   0.3659668  -0.01296234 -0.7324219  -1.1523438 ] 1   2 
[-0.73828125  0.5986328  -0.08850098 -0.08685303  0.06185913  0.20727539] 1   1 Match 60

[-0.3395996   0.2364502   0.37646484  0.5473633  -0.2956543  -0.9165039 ] 3   3 Ma

[-1.4814453  -0.08538818  0.5810547   0.56689453  0.19750977  0.0569458 ] 2   3 
[-0.74121094  1.4736328  -0.85498047  0.48950195 -0.8691406  -0.4741211 ] 1   5 
[-1.3632812  -0.43603516 -0.32714844  0.36035156  0.9194336   0.48120117] 4   4 Match 91

[ 0.33569336  0.5385742   0.27661133  0.04528809 -0.7246094  -1.0839844 ] 1   1 Match 92

[-0.23632812  0.50146484  0.05355835 -0.17102051 -0.2746582  -0.16210938] 1   2 
[-1.4824219  -0.3552246  -0.56103516  0.23376465  1.2597656   0.77978516] 4   4 Match 93

[-1.2060547   0.2824707   0.1665039   0.42041016  0.09661865 -0.0836792 ] 3   1 
[-0.21557617  0.5991211   0.03424072 -0.02685547 -0.10516357 -0.37475586] 1   1 Match 94

[-0.4272461   1.8603516  -0.74560547  0.4897461  -1.0410156  -0.9399414 ] 1   1 Match 95

[ 0.06512451  0.40576172  0.43188477  0.15576172 -0.6298828  -1.0527344 ] 2   2 Match 96

[-1.3271484  -0.02348328  0.06921387 -0.15222168  0.4741211   0.8305664 ] 5   0 
[-0.07397461  0.5683594   0.19897461 -0.18383789 -0.421


[-1.4726562  -0.46044922 -0.12902832  0.2578125   0.96191406  0.54833984] 4   5 
[-0.69091797  0.5961914  -0.04321289 -0.20385742 -0.04330444  0.44335938] 1   1 Match 138

[-1.5791016  -0.38500977 -0.7211914   0.18322754  1.3222656   0.90722656] 4   3 
[-0.5698242   0.46875     0.5395508   0.46923828 -0.35888672 -0.7421875 ] 2   3 
[-1.0966797   0.36206055  0.1583252   0.31762695 -0.08843994  0.04302979] 1   3 
[-1.0576172  -0.04318237  0.13171387  0.03863525  0.14562988  0.19128418] 5   3 
[-0.6123047  -0.0047493   0.43676758  0.43481445 -0.32983398 -0.7290039 ] 2   4 
[-1.2089844  -0.4555664  -0.04421997 -0.14404297  0.796875    0.64941406] 4   3 
[-0.2685547   0.07617188  0.21850586  0.7949219  -0.60009766 -1.1240234 ] 3   3 Match 139

[-0.63378906  0.17565918  0.4658203   0.3876953  -0.2932129  -0.8798828 ] 2   5 
[-1.6240234  -0.1373291  -0.12976074  0.78466797  0.7241211  -0.0100708 ] 3   4 
[-1.5351562   0.14831543 -0.1739502   0.44067383  0.7832031   0.7885742 ] 5   5 Match 14

...

[-1.7021484  -0.42333984 -0.15734863  0.45922852  1.0703125   0.53808594] 4   5 
[-1.1152344   0.16723633  0.31152344  0.2142334   0.04608154  0.13879395] 2   2 Match 193

[-0.5209961   0.4116211   0.01319122  0.0760498  -0.03207397 -0.12609863] 1   2 
[-0.90625     0.34472656  0.2006836   0.33789062 -0.0802002  -0.10394287] 1   4 
[-1.6064453   0.08294678 -0.35864258  0.6748047   0.7158203   0.5932617 ] 4   1 
[-1.6279297  -0.20410156 -0.02090454  0.28149414  0.9296875   0.6254883 ] 4   4 Match 194

[-0.02355957  0.4567871   0.01454163 -0.02694702 -0.1270752  -0.2578125 ] 1   3 
[-1.8105469  -0.39233398 -0.4086914   0.86572266  1.1962891   0.46679688] 4   4 Match 195

[-1.0830078   1.9189453  -0.5649414   0.37695312 -0.796875   -0.5883789 ] 1   5 
[-0.6147461   0.5234375   0.21289062  0.20739746 -0.19763184 -0.6591797 ] 1   2 
[-1.7246094  -0.18347168 -0.18811035  0.7866211   0.78027344  0.42138672] 3   5 
[-1.2851562   0.25463867 -0.22216797  0.45996094  0.28344727  0.29223633] 3   3

[-0.39672852 -0.02127075  0.5097656   0.4423828  -0.47998047 -1.0332031 ] 2   4 
[-1.1572266   0.081604    0.15307617  0.64501953 -0.074646   -0.15600586] 3   5 
[-1.7666016  -0.34423828 -0.44140625  0.4868164   1.2333984   0.78466797] 4   4 Match 235

[-1.7666016   0.0453186   0.2355957   0.7998047   0.5415039   0.08990479] 3   3 Match 236

[-0.84277344  0.03390503  0.50390625  0.4482422  -0.17199707 -0.5463867 ] 2   0 
[-0.10345459  0.37597656  0.5957031   0.36523438 -0.5654297  -1.1650391 ] 2   2 Match 237

[-1.6513672  -0.40405273 -0.45239258  0.40185547  1.1376953   0.85302734] 4   4 Match 238

[ 0.30981445  0.5942383   0.39697266 -0.08349609 -0.67089844 -0.93603516] 1   4 
[-1.5878906  -0.17016602 -0.44262695  0.30371094  1.03125     0.9916992 ] 4   2 
[-1.7490234  -0.39916992 -0.16271973  0.6245117   0.96240234  0.47338867] 4   3 
[-1.7861328  -0.41870117 -0.24865723  0.39111328  1.2792969   0.8076172 ] 4   5 
[-1.2705078  -0.09033203  0.42358398  0.5864258   0.25439453 -0.38598

[-1.9189453  -0.38916016 -0.20654297  0.8574219   1.0839844   0.3395996 ] 4   4 Match 267

[ 0.06604004  0.45141602  0.27270508 -0.17480469 -0.5058594  -0.67578125] 1   3 
[-0.6591797   0.05773926  0.5058594   0.3515625  -0.10229492 -0.67089844] 2   3 
[-0.49804688  0.1829834   0.4506836   0.23999023 -0.3972168  -0.5029297 ] 2   4 
[-1.484375   -0.05584717  0.3071289   0.9995117   0.0838623  -0.57666016] 3   3 Match 268

[ 0.46728516  0.43310547  0.23754883  0.11303711 -0.67529297 -1.1601562 ] 0   1 
[ 0.02633667  0.76416016  0.40039062 -0.09069824 -0.6484375  -0.79833984] 1   1 Match 269

[-0.91308594  0.3647461  -0.08184814 -0.07873535  0.16699219  0.5566406 ] 5   1 
[-1.5673828   0.18249512 -0.20788574  0.4892578   0.65478516  0.56347656] 4   5 
[ 0.2529297   0.578125    0.46118164 -0.06268311 -0.6923828  -0.97265625] 1   0 
[-1.7451172  -0.6176758  -0.02578735  0.98339844  0.74658203  0.33984375] 3   1 
[-1.2480469  -0.0925293   0.22961426  0.26293945  0.25927734  0.08825684] 3   3

[-1.5634766   0.10455322 -0.44018555  0.24572754  0.9760742   0.9057617 ] 4   4 Match 301

[-0.32128906  0.38012695 -0.06817627 -0.07330322 -0.13708496  0.0713501 ] 1   4 
[ 0.12249756  0.6713867   0.27539062  0.22961426 -0.6459961  -1.1689453 ] 1   4 
[-0.49536133  0.08477783  0.01580811  0.10931396  0.02684021 -0.19018555] 3   3 Match 302

[-1.1865234   0.2019043   0.05648804  0.37939453  0.3310547   0.23547363] 3   4 
[-1.328125    0.00647736 -0.39135742  0.43920898  0.8022461   0.5439453 ] 4   4 Match 303

[-0.5756836  -0.04266357  0.20092773  0.07080078 -0.0378418  -0.18896484] 2   2 Match 304

[-1.5742188  -0.09387207 -0.10437012  0.1373291   0.90234375  0.6777344 ] 4   5 
[-0.6425781  -0.07275391  0.5371094   0.5571289  -0.26416016 -0.96435547] 3   4 
[ 0.23120117  0.48876953  0.453125    0.27807617 -0.67626953 -1.2783203 ] 1   1 Match 305

[-0.2849121   0.5175781   0.31982422  0.06512451 -0.51464844 -0.44799805] 1   1 Match 306

[-1.6074219  -0.2685547  -0.27563477  0.6635742  


[-0.68603516  0.5004883  -0.21813965 -0.2310791   0.171875    0.3544922 ] 1   4 
[-1.7265625  -0.17407227 -0.19030762  0.57470703  0.9326172   0.65625   ] 4   2 
[-1.5126953  -0.15991211 -0.32128906  0.14343262  0.9819336   0.89746094] 4   5 
[-1.2939453   0.00794983  0.08239746  0.7597656   0.30029297 -0.3005371 ] 3   5 
[-0.3125      0.0581665   0.24291992  0.35351562 -0.29345703 -0.8779297 ] 3   2 
[-0.8696289   0.17260742  0.05053711  0.12426758  0.11676025  0.14562988] 1   2 
[-1.3359375  -0.02610779  0.17089844  1.0595703   0.15209961 -0.7270508 ] 3   4 
[-1.5986328  -0.14575195 -0.38916016  0.2993164   0.90722656  0.83154297] 4   4 Match 330

[-0.01739502  0.5004883   0.25756836 -0.04855347 -0.47680664 -0.46777344] 1   1 Match 331

[ 0.3815918   0.5888672   0.36132812 -0.14916992 -0.7182617  -0.9243164 ] 1   0 
[-1.5751953  -0.25610352 -0.22302246  0.26049805  1.0332031   0.5444336 ] 4   4 Match 332

[-1.1064453  -0.11901855  0.37817383  0.66308594  0.0994873  -0.56933594] 3   

In [118]:
from sklearn import metrics
 
print(metrics.confusion_matrix(test['labels'],Pred))

[[  1  45  15  12  11   8]
 [  3 112  18  55  42  20]
 [  1  71  33  60  35  14]
 [  0  69  27  87  65  19]
 [  0  42  16  74  90  27]
 [  1  39   9  47  82  33]]


In [119]:
target_names = ['Pants', 'False', 'Barely-True','Half-True','Mostly-True','True']

print(metrics.classification_report(test['labels'], Pred,target_names =target_names))

              precision    recall  f1-score   support

       Pants       0.17      0.01      0.02        92
       False       0.30      0.45      0.36       250
 Barely-True       0.28      0.15      0.20       214
   Half-True       0.26      0.33      0.29       267
 Mostly-True       0.28      0.36      0.31       249
        True       0.27      0.16      0.20       211

    accuracy                           0.28      1283
   macro avg       0.26      0.24      0.23      1283
weighted avg       0.27      0.28      0.26      1283



In [120]:
Fold_Predictions=pd.DataFrame(Pred, columns=['Pred4'] )
Fold_Predictions

,Pred4
0,4
1,2
2,1
3,3
4,4
...,...
1278,4
1279,1
1280,0
1281,2


In [121]:
Fold_Predictions.to_excel(output_folder+'/Saves/fold4_Predictions.xls')

In [122]:
#clearing GPU cache

del(model)
del(TrainResult, TrainModel_outputs, EvalResult, EvalModel_outputs, TestResult, TestModel_outputs, wrong_predictions)
torch.cuda.empty_cache()

## Fold 5: training & capturing predictions

In [123]:
fold_number='5'

train=pd.read_excel('./folds/train_fold'+fold_number+'.xls')
Eval=pd.read_excel('./folds/valid'+fold_number+'.xls') #evaluation set


In [125]:
 
output_folder='./folds/fold'+fold_number+'/'+model_class+'/'+model_version+"/"
cache_directory= "./folds/fold"+fold_number+'/'+model_class+"/"+model_version+"/cache/"


print('model variables were set up: ')

 
save_every_steps=1285
# assuming training batch size of 8
# any number above 1284 saves the model only at every epoch
# Saving the model mid training very often will consume disk space fast

train_args={
    "output_dir":output_folder,
    "cache_dir":cache_directory,
    'reprocess_input_data': True,
    'overwrite_output_dir': True,
    'num_train_epochs': 2,
    "save_steps": save_every_steps,
    "learning_rate": 2e-5,
    "train_batch_size": 64,
    "eval_batch_size": 16,
    "adam_epsilon": 1e-8,
    "evaluate_during_training_steps": 312,
    "max_seq_length": 100,
    "n_gpu": 1,
}

# Create a ClassificationModel
model = ClassificationModel(model_class, model_version, num_labels=labels_count, args=train_args) 

model variables were set up: 


In [59]:
# loading the checkpoint that gave the best result

CheckPoint='checkpoint-       1'   


preSavedCheckpoint=output_folder+CheckPoint

print('Loading model, please wait...')
model = ClassificationModel( model_class, preSavedCheckpoint, num_labels=labels_count, args=train_args) 
print('model in use is :', preSavedCheckpoint )


Loading model, please wait...
model in use is : ./folds/fold5/bert/bert-base-cased/checkpoint-143-epoch-1


In [126]:
# Train the model
current_time = datetime.now()
model.train_model(train)
print("Training time: ", datetime.now() - current_time)

Converting to features started. Cache is not used.



Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic


Running loss: 1.744900


Running loss: 1.636861

Training of bert model complete. Saved to ./folds/fold5/bert/bert-base-cased/.
Training time:  0:07:44.510240


In [127]:
TrainResult, TrainModel_outputs, wrong_predictions = model.eval_model(train, acc=sklearn.metrics.accuracy_score)
 
EvalResult, EvalModel_outputs, wrong_predictions = model.eval_model(Eval, acc=sklearn.metrics.accuracy_score)

TestResult, TestModel_outputs, wrong_predictions = model.eval_model(test, acc=sklearn.metrics.accuracy_score)

print('Training Result:', TrainResult['acc'])
#print('Model Out:', TrainModel_outputs)

print('Eval Result:', EvalResult['acc'])
#print('Model Out:', EvalModel_outputs)

print('Test Set Result:', TestResult['acc'])
#print('Model Out:', TestModel_outputs)

Converting to features started. Cache is not used.



{'mcc': 0.15802339077127905, 'acc': 0.3145484045429962, 'eval_loss': 1.6222220193143535}
Converting to features started. Cache is not used.



{'mcc': 0.10277400430038135, 'acc': 0.27123050259965337, 'eval_loss': 1.6842008566034252}
Converting to features started. Cache is not used.



{'mcc': 0.08702060481441222, 'acc': 0.2603273577552611, 'eval_loss': 1.672672732376758}
Training Result: 0.3145484045429962
Eval Result: 0.27123050259965337
Test Set Result: 0.2603273577552611


In [128]:
Pred=[]

countCorrect=0

for row in range(TestModel_outputs.shape[0]):
    outputs=TestModel_outputs[row]
    #print(test.iloc[row,0])
    print(outputs, end=' ')
    
    result=0
    if outputs[0]<outputs[1]:result=1
    if outputs[result]<outputs[2]:result=2
    if outputs[result]<outputs[3]:result=3
    if outputs[result]<outputs[4]:result=4
    if outputs[result]<outputs[5]:result=5
    Pred.append(result)
    print(result, ' ',test.iloc[row,1], end=' ')
    if result==test.iloc[row,1]:
        countCorrect+=1
        print('Match',countCorrect)
    print('')

print(countCorrect)
#Pred

[-1.9833984  -0.41357422 -0.6977539   0.4482422   1.0449219   0.62939453] 4   4 Match 1

[-0.6152344   0.05007935  0.06335449  0.19702148 -0.38623047 -0.9199219 ] 3   3 Match 2

[-0.57910156  0.36938477 -0.05984497  0.06378174  0.00885773 -0.03781128] 1   1 Match 3

[-1.0390625   0.10284424 -0.16467285  0.08721924 -0.15515137 -0.23706055] 1   5 
[-1.8476562  -0.3010254  -0.5727539   0.6533203   0.5185547   0.11834717] 3   5 
[-1.1416016   0.14709473 -0.31811523  0.1204834   0.11584473  0.15905762] 5   2 
[-1.9179688  -0.26635742 -0.55322266  0.47045898  0.8691406   0.2130127 ] 4   4 Match 4

[-1.0673828  -0.2434082  -0.14697266  0.17126465  0.02354431 -0.52978516] 3   5 
[-1.3076172   0.25       -0.4921875   0.01499176  0.7363281   0.8652344 ] 5   4 
[-1.5302734  -0.26123047 -0.4489746   0.37524414  0.4892578   0.07885742] 4   5 
[-0.6254883  -0.07299805 -0.11071777  0.08966064 -0.42895508 -0.8510742 ] 3   5 
[-1.0458984   0.21875    -0.19384766  0.23474121  0.00973511 -0.21362305] 3  

[-1.5839844  -0.17199707 -0.35107422  0.45922852  0.20617676 -0.19946289] 3   4 
[-0.78027344  1.6933594  -0.72509766  0.36889648 -1.0175781  -0.40429688] 1   1 Match 28

[-0.57666016  0.2290039   0.21374512  0.08770752 -0.62060547 -0.95458984] 1   0 
[-0.6982422   0.23657227 -0.51464844 -0.36499023  0.10528564  0.15563965] 1   5 
[-1.5683594  -0.296875   -0.2980957   0.5078125   0.20959473 -0.16149902] 3   1 
[-0.8696289   0.23120117  0.08972168  0.13757324 -0.5541992  -0.8696289 ] 1   1 Match 29

[-1.5537109  -0.33691406 -0.40942383  0.3791504   0.5131836   0.02650452] 4   3 
[-0.9086914  -0.16027832 -0.10443115  0.07727051 -0.12390137 -0.47216797] 3   2 
[-0.96533203  0.6826172  -0.00780106  0.09686279 -0.34545898 -0.44580078] 1   2 
[-1.8339844  -0.36450195 -0.61279297  0.2548828   0.9350586   0.97558594] 5   4 
[-1.4316406  -0.10076904 -0.4050293   0.02520752  0.5214844   0.60546875] 5   4 
[-1.0722656   0.56396484  0.02043152 -0.07165527 -0.3400879  -0.1204834 ] 1   5 
[-0.386962

[-0.4963379   0.71777344  0.14624023 -0.11242676 -0.6484375  -0.65722656] 1   0 
[-1.5908203  -0.18005371 -0.41455078  0.26220703  0.4111328   0.22521973] 4   3 
[-1.296875   -0.3244629  -0.39135742  0.35913086  0.3618164  -0.01585388] 4   1 
[-0.08746338  0.51171875  0.3400879  -0.01551056 -0.79589844 -1.0605469 ] 1   0 
[-0.91015625  0.16271973 -0.2939453  -0.09503174  0.18322754  0.3474121 ] 5   0 
[-1.5996094  -0.23840332 -0.33691406  0.3647461   0.22351074 -0.07202148] 3   1 
[-0.20727539  0.37670898  0.03652954  0.05303955 -0.7519531  -1.0439453 ] 1   3 
[-0.83740234  0.12524414  0.05444336  0.20092773 -0.39819336 -0.67578125] 3   5 
[ 0.4675293   0.38330078  0.24511719 -0.30419922 -0.63964844 -0.28686523] 0   0 Match 57

[-1.9433594  -0.4230957  -0.59765625  0.36157227  0.82128906  0.5698242 ] 4   4 Match 58

[-0.8642578  -0.02159119  0.26782227  0.18981934 -0.59521484 -1.0791016 ] 2   2 Match 59

[ 0.5541992   0.46313477  0.49926758 -0.38183594 -0.67041016 -0.5625    ] 0   2 
[

[-0.4230957   0.12481689  0.00161552 -0.02987671 -0.46240234 -0.41845703] 1   2 
[ 0.27124023  0.70996094  0.30615234 -0.28100586 -0.39575195 -0.2487793 ] 1   3 
[-0.5732422   0.48510742 -0.04098511 -0.15759277 -0.25927734 -0.1538086 ] 1   0 
[-1.6484375  -0.42163086 -0.5957031   0.31640625  0.53271484  0.32128906] 4   4 Match 86

[-0.8808594   0.11437988 -0.07983398  0.00971222 -0.23535156 -0.18786621] 1   1 Match 87

[-1.5488281  -0.20410156 -0.57666016  0.171875    0.5913086   0.4567871 ] 4   3 
[-1.3798828   0.10174561  0.03018188  0.11621094 -0.08459473 -0.12451172] 3   3 Match 88

[-0.94873047  1.7773438  -0.73876953  0.39794922 -0.8886719  -0.41308594] 1   5 
[-1.0976562  -0.08825684 -0.30200195 -0.01360321  0.07910156 -0.06561279] 4   4 Match 89

[-0.24645996  0.4013672  -0.06616211 -0.16699219 -0.6015625  -0.8251953 ] 1   1 Match 90

[-1.1845703  -0.23083496 -0.17675781  0.16149902  0.11706543 -0.5444336 ] 3   2 
[-1.6279297  -0.2590332  -0.53125     0.05944824  0.79003906  0.

[-0.18225098  0.65771484  0.16882324 -0.20324707 -0.52441406 -0.38500977] 1   3 
[-1.1123047  -0.08917236  0.02848816  0.14611816  0.1295166  -0.21289062] 3   3 Match 123

[-0.8666992   0.28686523  0.16210938  0.23608398 -0.1784668  -0.38867188] 1   4 
[-0.86035156  0.09674072  0.10040283  0.22668457 -0.32348633 -0.7529297 ] 3   3 Match 124

[-0.74609375 -0.16381836  0.05993652  0.32763672 -0.35986328 -0.984375  ] 3   3 Match 125

[-0.84375    -0.15197754  0.05307007  0.2836914  -0.24584961 -0.95166016] 3   5 
[-1.2792969  -0.21228027 -0.32666016  0.39257812  0.5463867   0.11633301] 4   4 Match 126

[-1.9033203  -0.25976562 -0.5214844   0.40112305  0.45947266  0.16699219] 4   5 
[-0.69873047  0.52490234 -0.17883301 -0.06994629 -0.06896973  0.20910645] 1   1 Match 127

[-0.5751953   0.05697632  0.23937988  0.10552979 -0.5571289  -1.1474609 ] 2   2 Match 128

[-0.9941406   0.1352539  -0.09344482  0.18920898 -0.22436523 -0.16223145] 3   4 
[-1.2294922  -0.05981445 -0.15576172  0.36376953 

...

[-1.6757812  -0.2932129  -0.41479492  0.5097656   0.21008301 -0.06622314] 3   3 Match 183

[-0.8935547   0.15185547 -0.0044632   0.20056152 -0.20898438 -0.7602539 ] 3   0 
[-1.1650391  -0.20812988 -0.3137207   0.36279297  0.16845703 -0.30444336] 3   2 
[-1.5546875  -0.37963867 -0.40161133  0.4008789   0.4169922  -0.05200195] 4   4 Match 184

[-1.1943359  -0.04760742 -0.45947266  0.14782715  0.26123047  0.38500977] 5   4 
[-1.0966797   0.19372559 -0.07189941  0.41064453  0.24475098  0.03065491] 3   4 
[-1.2451172   0.12286377 -0.2998047   0.13293457  0.43969727  0.13977051] 4   4 Match 185

[-1.1181641  -0.090271   -0.08770752  0.16223145  0.04043579 -0.4638672 ] 3   3 Match 186

[-1.2041016   0.0569458   0.02696228  0.40893555 -0.04995728 -0.5888672 ] 3   0 
[-1.6708984  -0.12927246 -0.37426758  0.3239746   0.5625     -0.03387451] 4   5 
[-0.8647461   0.4946289   0.04632568  0.04693604 -0.34057617 -0.4326172 ] 1   2 
[-0.81347656 -0.00909424 -0.25024414  0.15881348 -0.15563965 -0.39038

[-1.6103516  -0.23425293 -0.41552734  0.40844727  0.7524414   0.26660156] 4   4 Match 222

[-1.25        0.2097168  -0.3347168  -0.01637268  0.4802246   0.79785156] 5   3 
[-0.75097656 -0.10412598 -0.02339172  0.17858887 -0.4165039  -0.9243164 ] 3   1 
[-2.03125    -0.27441406 -0.92089844  0.37036133  0.9433594   0.8779297 ] 4   3 
[-0.8876953   0.3017578  -0.33911133 -0.17944336  0.11700439  0.3239746 ] 5   4 
[-1.5068359  -0.23657227 -0.56933594  0.18786621  0.7402344   0.8051758 ] 5   4 
[-1.1855469  -0.22058105 -0.25708008  0.29516602 -0.06945801 -0.46166992] 3   5 
[-0.6147461   0.33251953 -0.20129395 -0.15600586  0.2800293   0.22180176] 1   4 
[-1.5302734  -0.23376465 -0.36889648  0.41186523  0.0287323  -0.07635498] 3   4 
[-1.0419922  -0.21557617 -0.19750977  0.2607422  -0.0539856  -0.55322266] 3   5 
[-0.25341797  0.46264648 -0.04263306 -0.26367188 -0.4260254  -0.1973877 ] 1   5 
[-0.85839844  0.03033447  0.06884766  0.1628418  -0.37402344 -0.53222656] 3   4 
[-0.57666016  0.11

[-1.1875     -0.20483398  0.05593872  0.28173828 -0.26464844 -0.56347656] 3   4 
[-0.18041992  0.24731445  0.16430664 -0.07171631 -0.5024414  -0.58691406] 1   4 
[-0.13391113  0.05233765  0.07769775  0.09362793 -0.5913086  -1.1259766 ] 3   2 
[-1.4472656  -0.04592896 -0.45825195 -0.01488495  0.54052734  0.7480469 ] 5   4 
[-0.40722656  0.15734863 -0.07470703 -0.15600586 -0.5234375  -0.80810547] 1   4 
[-1.8447266  -0.4399414  -0.5703125   0.35058594  0.65771484  0.5493164 ] 4   4 Match 249

[-1.3457031   0.13964844 -0.4296875  -0.01754761  0.75634766  0.59228516] 4   5 
[-0.75        0.67822266 -0.28027344 -0.19506836  0.29516602  0.5332031 ] 1   5 
[-1.5722656  -0.2529297  -0.29174805  0.2548828   0.46264648  0.1784668 ] 4   0 
[-1.8427734  -0.1463623  -0.44628906  0.3071289   0.7734375   0.7207031 ] 4   5 
[-1.3535156  -0.3671875  -0.43847656  0.29077148  0.34423828 -0.18200684] 4   3 
[-1.5654297  -0.05441284 -0.32910156  0.23901367  0.4111328   0.10546875] 4   4 Match 250

[ 0.0368

[-1.5195312 -0.2064209 -0.59375    0.2553711  0.7680664  0.8144531] 5   5 Match 280

[-0.9658203  -0.09161377  0.02514648  0.22705078 -0.11462402 -0.77685547] 3   4 
[-0.8486328  -0.10992432  0.14221191  0.22180176 -0.1665039  -0.6894531 ] 3   4 
[-1.1103516  -0.21911621 -0.23083496  0.30004883  0.28027344 -0.4177246 ] 3   1 
[-0.32373047  0.53515625 -0.10491943 -0.15844727 -0.02618408  0.16821289] 1   0 
[-0.31713867  0.49072266 -0.20495605 -0.2800293   0.11462402  0.5620117 ] 5   4 
[-1.7626953  -0.1697998  -0.4868164   0.39501953  0.3239746   0.19921875] 3   3 Match 281

[-0.46069336 -0.00765991  0.12512207  0.09259033 -0.546875   -1.1708984 ] 2   3 
[-0.39404297  0.31982422 -0.02404785 -0.0645752  -0.5283203  -0.6845703 ] 1   1 Match 282

[-1.9384766  -0.39501953 -0.7163086   0.34179688  1.0126953   0.87939453] 4   4 Match 283

[-1.1455078   0.14404297 -0.05825806  0.42211914 -0.27978516 -0.53466797] 3   2 
[-1.1367188  -0.06384277 -0.3190918   0.31420898  0.15307617 -0.27490234] 3

[-0.11065674  0.29077148 -0.06167603 -0.01982117 -0.5541992  -0.8251953 ] 1   0 
[-0.24987793  0.4633789   0.11083984 -0.14904785 -0.3334961   0.05987549] 1   1 Match 310

[-1.2128906   0.0604248  -0.25463867  0.23034668  0.37548828  0.296875  ] 4   4 Match 311

[-0.20666504  0.23693848 -0.04086304 -0.03344727 -0.34960938 -0.7841797 ] 1   2 
[-0.46264648  0.28466797 -0.18518066 -0.1685791  -0.29052734  0.11706543] 1   4 
[-1.3359375  -0.2590332  -0.22961426  0.20385742  0.35205078  0.18188477] 4   2 
[ 0.24597168  0.4580078   0.11920166 -0.33740234 -0.67871094 -0.3984375 ] 1   5 
[-1.7470703  -0.43164062 -0.4963379   0.34936523  0.69921875  0.48999023] 4   2 
[-1.2675781  -0.33398438 -0.2783203   0.30737305  0.21374512 -0.24328613] 3   2 
[-0.63427734  0.19262695 -0.12219238 -0.15881348 -0.02282715  0.06695557] 1   1 Match 312

[-0.97998047  0.24829102 -0.02474976  0.23217773 -0.19909668 -0.7158203 ] 1   5 
[ 0.01242828  0.5371094   0.05239868 -0.17114258 -0.60595703 -0.6845703 ] 1   4

In [129]:
from sklearn import metrics
 
print(metrics.confusion_matrix(test['labels'],Pred))

[[  3  54   9   9  14   3]
 [  2 111   9  50  57  21]
 [  4  75  24  60  37  14]
 [  3  93  11  84  67   9]
 [  1  61   4  63  80  40]
 [  1  49   2  44  83  32]]


In [130]:
target_names = ['Pants', 'False', 'Barely-True','Half-True','Mostly-True','True']

print(metrics.classification_report(test['labels'], Pred,target_names =target_names))

              precision    recall  f1-score   support

       Pants       0.21      0.03      0.06        92
       False       0.25      0.44      0.32       250
 Barely-True       0.41      0.11      0.18       214
   Half-True       0.27      0.31      0.29       267
 Mostly-True       0.24      0.32      0.27       249
        True       0.27      0.15      0.19       211

    accuracy                           0.26      1283
   macro avg       0.27      0.23      0.22      1283
weighted avg       0.28      0.26      0.24      1283



In [131]:
Fold_Predictions=pd.DataFrame(Pred, columns=['Pred5'] )
Fold_Predictions

,Pred5
0,4
1,3
2,1
3,1
4,3
...,...
1278,4
1279,1
1280,1
1281,1


In [132]:
Fold_Predictions.to_excel(output_folder+'/Saves/fold5_Predictions.xls')

In [133]:
#clearing GPU cache

del(model)
del(TrainResult, TrainModel_outputs, EvalResult, EvalModel_outputs, TestResult, TestModel_outputs, wrong_predictions)
torch.cuda.empty_cache()

# Comparing the Predictions

In [134]:
Pred1=pd.read_excel('./folds/fold1/bert/bert-base-cased/'+'Saves/fold1_Predictions.xls')
Pred2=pd.read_excel('./folds/fold2/bert/bert-base-cased/'+'Saves/fold2_Predictions.xls')
Pred3=pd.read_excel('./folds/fold3/bert/bert-base-cased/'+'Saves/fold3_Predictions.xls')
Pred4=pd.read_excel('./folds/fold4/bert/bert-base-cased/'+'Saves/fold4_Predictions.xls')
Pred5=pd.read_excel('./folds/fold5/bert/bert-base-cased/'+'Saves/fold5_Predictions.xls')


In [135]:
 for row in range(len(Pred1)):
        
        print(Pred1.iloc[row,1] , end=',')
        print(Pred2.iloc[row,1]  , end=',')
        print(Pred3.iloc[row,1] , end=',')
        print(Pred4.iloc[row,1] , end=',')
        print(Pred5.iloc[row,1] )
    

4,4,4,4,4
1,1,1,2,3
1,1,3,1,1
3,3,3,3,1
3,4,4,4,3
5,5,4,1,5
4,4,4,4,4
4,4,3,3,3
5,5,5,5,5
4,3,3,3,4
1,3,1,3,3
3,3,3,1,3
1,1,1,1,1
1,1,1,1,1
3,5,4,4,5
1,1,1,1,1
4,5,4,5,4
3,4,3,4,1
3,3,3,3,3
3,3,3,3,4
1,1,5,5,4
4,4,4,4,4
5,5,4,5,5
3,3,2,3,3
1,1,4,5,1
1,5,3,5,1
3,3,4,3,1
4,4,3,3,1
3,1,4,4,3
1,1,1,5,1
1,3,3,2,1
4,5,4,5,5
1,1,1,1,1
3,3,3,4,3
1,5,1,1,4
3,3,3,4,1
5,5,5,5,5
2,1,1,1,1
3,5,4,3,3
3,5,1,3,3
2,5,3,4,3
1,3,1,4,1
5,5,1,1,1
2,1,1,2,1
1,5,1,5,1
2,2,1,1,3
4,5,4,5,5
1,1,2,2,1
4,4,4,4,1
4,5,4,5,3
4,5,4,4,4
3,5,4,4,5
3,4,4,3,4
1,5,5,3,1
5,5,1,5,5
2,2,1,2,3
4,4,5,4,5
3,3,5,5,3
5,5,5,5,5
1,1,1,1,3
1,1,1,1,1
3,5,4,5,5
1,1,1,1,1
1,1,3,3,1
1,1,1,1,3
3,3,4,5,3
4,5,4,4,3
4,5,4,4,4
3,1,1,1,3
3,5,4,4,3
3,3,1,3,4
1,5,3,5,1
1,1,1,3,3
1,1,4,5,1
1,1,1,1,1
1,1,1,1,1
3,3,4,4,5
1,1,1,1,1
5,5,5,5,3
3,3,3,3,3
3,3,4,4,3
1,5,1,4,4
3,5,4,5,1
4,4,3,3,4
1,1,1,1,1
2,1,5,3,3
3,1,3,1,1
2,1,3,3,1
1,1,1,2,1
3,3,3,3,3
1,1,1,1,3
1,1,1,1,1
0,1,0,1,1
4,4,4,4,4
3,3,4,3,4
1,1,1,1,1
1,1,1,1,1
1,5,5,1,3
5,1,4,1,3
2,1,2,2,2


3,1,1,3,1
1,1,2,1,1
3,5,4,4,3
3,4,3,3,3
2,3,1,2,2
3,5,1,1,1
3,3,3,3,3
1,1,3,4,1
3,3,4,3,4
1,1,2,2,1
3,3,3,4,2
3,3,4,3,3
1,4,3,1,1
1,1,1,1,1
3,4,3,3,3
5,5,1,5,5
5,4,4,3,4
4,4,4,5,4
1,1,1,1,1
4,4,4,4,5
3,3,1,3,3
1,1,1,1,1
3,3,3,3,3
1,1,1,1,1
1,1,1,1,1
0,2,0,1,1
4,4,4,4,4
4,3,3,4,4
1,1,1,1,1
1,1,1,1,1
5,5,4,5,5
3,4,1,4,3
4,4,4,4,4
3,3,3,4,3
4,4,4,4,4
1,1,1,1,1
1,1,1,1,1
1,1,1,1,2
4,3,3,4,4
1,1,3,1,1
4,4,4,4,5
4,4,4,4,4
3,5,4,4,4
3,3,1,3,4
3,3,1,2,3
1,2,2,1,1
2,2,2,2,2
1,5,1,1,5
3,5,4,2,5
1,1,1,1,1
5,4,3,3,1
5,4,1,1,1
3,3,1,2,2
4,4,4,4,4
4,4,4,4,5
1,3,1,1,1
3,3,4,4,4
1,1,1,1,1
3,3,3,3,3
1,2,2,3,1
1,2,1,3,3
5,5,4,5,5
1,1,1,1,1
4,3,4,4,4
5,4,4,5,4
5,5,5,5,1
4,4,4,4,4
4,4,4,4,4
4,4,3,3,4
4,4,4,4,4
1,1,1,1,1
3,3,1,2,1
3,3,1,2,3
3,3,3,3,3
1,1,1,0,1
1,4,1,1,1
5,5,5,5,5
4,4,4,4,4
1,1,1,1,1
3,3,4,3,3
3,3,1,3,3
5,5,3,1,5
1,1,1,1,1
5,4,1,1,5
4,5,4,3,4
2,2,1,1,3
4,3,3,3,3
3,3,2,3,3
4,1,1,1,4
3,5,4,4,4
1,1,5,1,1
3,1,1,1,1
1,1,1,1,1
4,4,4,4,4
2,3,1,2,2
1,1,3,1,1
4,5,4,5,5
4,4,4,4,4
4,4,4,4,4
2,1,1,2,3


In [136]:
results=pd.DataFrame( columns=['text', 'label','fold1','fold2','fold3','fold4','fold5'])

results['text']=test['text']
results['label']=test['labels']
results['fold1']=Pred1['Pred1'] 
results['fold2']=Pred2['Pred2'] 
results['fold3']=Pred3['Pred3'] 
results['fold4']=Pred4['Pred4'] 
results['fold5']=Pred5['Pred5'] 

        
results

,text,label,fold1,fold2,fold3,fold4,fold5
0,New Mexico was 46th in teacher pay (when he wa...,4,4,4,4,4,4
1,Barack Obama and Hillary Clinton have changed ...,3,1,1,1,2,3
2,I'll tell you what I can tell this country: If...,1,1,1,3,1,1
3,Tommy Thompson created the first school choice...,5,3,3,3,3,1
4,Fifty-six percent decline in overall crime. A ...,5,3,4,4,4,3
...,...,...,...,...,...,...,...
1278,"We have trade agreements with 20 countries, an...",1,4,4,4,4,4
1279,On Donald Trumps plan to cut federal funding t...,4,1,1,3,1,1
1280,"Black Lives Matter, who are attacking law enfo...",2,0,1,0,0,1
1281,Latina who enthusiastically supported Donald T...,0,1,1,2,2,1


In [137]:
now=datetime.now()

results.to_excel('./folds/'+model_class+'_results'+now.strftime("%m-%d-%Y %H-%M")+'.xls' ,index=False)